In [ ]:
import sys
import os
import asyncio
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QMessageBox, QScrollArea
from PyQt5.QtCore import QThread, pyqtSignal
import subprocess

# Set the event loop policy to WindowsSelectorEventLoopPolicy
if sys.platform.startswith('win'):
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

# Base directory path
BASE_DIR = r"C:\Users\bovta\Desktop\Aaron (Intern)\Aaron (Intern)\VS code Stuff\Final UI"

# Specify the full paths to your Python files here
SCRIPT_PATHS = {
    "IndividualChannel UI": os.path.join(BASE_DIR, "IndividualChannel UI.py"),
    "MultichannelCombined": os.path.join(BASE_DIR, "MultichannelCombined.py"),
    "Single Channel Sampling (Untied)": os.path.join(BASE_DIR, "single channel sampling (untied).py"),
    "Single Channel Sampling (Untied and Multichannel)": os.path.join(BASE_DIR, "single channel sampling (untied and multichannel sampling allowed).py"),
    "Optical Power Oscilloscope UI": os.path.join(BASE_DIR, "optical power oscilloscope UI.py")
}

class PythonScriptExecutor(QThread):
    finished = pyqtSignal(str)
    error = pyqtSignal(str)

    def __init__(self, script_path):
        super().__init__()
        self.script_path = script_path

    def run(self):
        try:
            subprocess.Popen([sys.executable, self.script_path], 
                             creationflags=subprocess.CREATE_NEW_CONSOLE)
            self.finished.emit(os.path.basename(self.script_path))
        except Exception as e:
            self.error.emit(str(e))

class DAQmxLauncher(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        # Create a scroll area
        scroll = QScrollArea()
        scroll.setWidgetResizable(True)
        scroll_content = QWidget()
        scroll_layout = QVBoxLayout(scroll_content)

        for script_name, script_path in SCRIPT_PATHS.items():
            btn = QPushButton(script_name, self)
            btn.clicked.connect(lambda checked, path=script_path: self.launch_script(path))
            scroll_layout.addWidget(btn)

        scroll_content.setLayout(scroll_layout)
        scroll.setWidget(scroll_content)
        layout.addWidget(scroll)

        self.setLayout(layout)
        self.setGeometry(300, 300, 400, 300)
        self.setWindowTitle('DAQmx Interface Launcher')
        self.show()

    def launch_script(self, script_path):
        if not os.path.exists(script_path):
            QMessageBox.critical(self, "Error", f"The file {script_path} does not exist.")
            return

        self.executor = PythonScriptExecutor(script_path)
        self.executor.finished.connect(self.on_script_finished)
        self.executor.error.connect(self.on_script_error)
        self.executor.start()

        QMessageBox.information(self, "Launching", f"Launching {os.path.basename(script_path)}. This may take a moment...")

    def on_script_finished(self, script_name):
        QMessageBox.information(self, "Success", f"Launched {script_name} successfully")

    def on_script_error(self, error_message):
        QMessageBox.critical(self, "Error", f"Failed to launch script: {error_message}")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = DAQmxLauncher()
    sys.exit(app.exec_())

In [ ]:
#Single channel
import sys
import numpy as np
import time
from functools import partial
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QHBoxLayout, QGridLayout, 
    QScrollArea, QComboBox, QLineEdit, QFileDialog, QMessageBox, QProgressDialog, QProgressBar
)
from PyQt5.QtCore import Qt, QThread, pyqtSignal, QRunnable, QThreadPool, QObject
import pyqtgraph as pg
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, AcquisitionType, Edge
from nidaqmx.stream_writers import AnalogMultiChannelWriter
from nidaqmx.stream_readers import AnalogMultiChannelReader


class WorkerSignals(QObject):
    finished = pyqtSignal()
    error = pyqtSignal(tuple)
    result = pyqtSignal(object)

class Worker(QRunnable):
    def __init__(self, fn, *args, **kwargs):
        super().__init__()
        self.fn = fn
        self.args = args
        self.kwargs = kwargs
        self.signals = WorkerSignals()

    def run(self):
        try:
            result = self.fn(*self.args, **self.kwargs)
        except:
            exctype, value = sys.exc_info()[:2]
            self.signals.error.emit((exctype, value, traceback.format_exc()))
        else:
            self.signals.result.emit(result)
        finally:
            self.signals.finished.emit()

class OutputThread(QThread):
    update_plot = pyqtSignal(str, np.ndarray, np.ndarray)
    error_occurred = pyqtSignal(str)
    finished = pyqtSignal()

    def __init__(self, channel, values, period, iterations, sample_rate):
        super().__init__()
        self.channel = channel
        self.values = values
        self.period = period
        self.iterations = iterations
        self.sample_rate = sample_rate

    def run(self):
        try:
            single_period_samples = len(self.values)
            total_samples = single_period_samples * self.iterations
            waveform = np.tile(self.values, self.iterations)
            
            # Calculate total duration based on period and iterations
            total_duration = self.period * self.iterations
            timestamps = np.linspace(0, total_duration, total_samples)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(self.channel)
                task.timing.cfg_samp_clk_timing(rate=self.sample_rate, sample_mode=AcquisitionType.FINITE, samps_per_chan=total_samples)
                
                task.write(waveform, auto_start=True)
                
                self.update_plot.emit(self.channel, timestamps, waveform)
                
                task.wait_until_done(timeout=total_duration + 5.0)
            
            self.finished.emit()
        except Exception as e:
            self.error_occurred.emit(str(e))
            
class DataProcessingThread(QThread):
    update_progress = pyqtSignal(int)
    update_plot = pyqtSignal(str, np.ndarray, np.ndarray, np.ndarray, np.ndarray)
    error_occurred = pyqtSignal(str)
    
    def __init__(self, channel, waveform, total_duration, ai_task, ao_task):
        super().__init__()
        self.channel = channel
        self.waveform = waveform
        self.total_duration = total_duration
        self.stop_event = threading.Event()

    def run(self):
        try:
            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO task
                ao_task.ao_channels.add_ao_voltage_chan(self.channel)
                ao_task.timing.cfg_samp_clk_timing(rate=len(self.waveform)/self.total_duration, 
                                                   sample_mode=AcquisitionType.FINITE, 
                                                   samps_per_chan=len(self.waveform))

                # Configure AI task
                ai_channel = f"Dev1/ai{self.channel[-1]}"  # Assumes channel format like "Dev1/ao0"
                ai_task.ai_channels.add_ai_voltage_chan(ai_channel)
                ai_task.timing.cfg_samp_clk_timing(rate=len(self.waveform)/self.total_duration, 
                                                   sample_mode=AcquisitionType.FINITE, 
                                                   samps_per_chan=len(self.waveform))

                # Set up synchronization
                ai_task.triggers.start_trigger.cfg_dig_edge_start_trig(f"/{self.channel}/StartTrigger")

                writer = AnalogMultiChannelWriter(ao_task.out_stream)
                reader = AnalogMultiChannelReader(ai_task.in_stream)

                writer.write_many_sample(np.array([self.waveform]))
                
                ao_task.start()
                ai_task.start()

                ao_data = self.waveform
                ai_data = np.zeros((1, len(self.waveform)))
                timestamps = np.linspace(0, self.total_duration, len(self.waveform))

                batch_size = 1000
                for i in range(0, len(self.waveform), batch_size):
                    if self.stop_event.is_set():
                        break
                    end = min(i + batch_size, len(self.waveform))
                    reader.read_many_sample(
                        ai_data[:, i:end], 
                        number_of_samples_per_channel=end - i,
                        timeout=5.0
                    )
                    progress = int((i / len(self.waveform)) * 100)
                    self.update_progress.emit(progress)
                    
                    self.update_plot.emit(self.channel, timestamps[:end], ao_data[:end], timestamps[:end], ai_data[0, :end])

                ao_task.wait_until_done(timeout=self.total_duration + 5.0)
                ai_task.wait_until_done(timeout=self.total_duration + 5.0)
                
                self.update_plot.emit(self.channel, timestamps, ao_data, timestamps, ai_data[0])
                self.update_progress.emit(100)

        except Exception as e:
            self.error_occurred.emit(str(e))
        finally:
            self.stop_event.set()

    def stop(self):
        self.stop_event.set()

class DAQmxController(QWidget):
    def __init__(self):
        super().__init__()
        self.threadpool = QThreadPool()

        self.ao_channels = ["Dev1/ao0", "Dev1/ao1", "Dev1/ao2", "Dev1/ao3"]
        self.ai_channels = ["Dev1/ai0", "Dev1/ai1", "Dev1/ai2", "Dev1/ai3"]
        self.terminal_configs = ["RSE", "Differential", "Pseudodifferential"]

        self.ao_values = [np.array([]) for _ in range(len(self.ao_channels))]
        self.ai_values = [np.array([]) for _ in range(len(self.ai_channels))]
        self.ao_timestamps = [np.array([]) for _ in range(len(self.ao_channels))]
        self.ai_timestamps = [np.array([]) for _ in range(len(self.ai_channels))]
        
        self.ao_period_textboxes = []
        self.ao_iterations_textboxes = []
        self.ao_file_paths = [None] * len(self.ao_channels)
        self.progress_bar = QProgressBar(self)
        self.progress_bar.setVisible(False)
        self.uploaded_data = {channel: None for channel in self.ao_channels}
        self.run_buttons = []
        self.ao_preview_plots = {}  # New dictionary for preview plots
        
        self.initUI()

    def initUI(self):
        main_layout = QVBoxLayout()
        control_layout = QGridLayout()
        control_layout.setHorizontalSpacing(2)
        control_layout.setVerticalSpacing(2)

        # Create a grid layout for the plots
        plot_layout = QGridLayout()
        plot_layout.setHorizontalSpacing(5)
        plot_layout.setVerticalSpacing(5)
        
        self.ao_labels = []
        self.ai_labels = []
        self.ao_plots = {}
        self.ai_plots = {}
        self.ai_terminal_configs = []
        self.ai_sampling_rates = []
        self.ai_min_voltages = []
        self.ai_max_voltages = []

        # Add "Run" buttons for each channel
        for i, ao_channel in enumerate(self.ao_channels):
            run_button = QPushButton("Run")
            run_button.clicked.connect(partial(self.run_ao_values, ao_channel))
            run_button.setEnabled(False)
            self.run_buttons.append(run_button)
            control_layout.addWidget(run_button, i, 9)
            
        # Create textboxes, labels, upload buttons, period, iterations fields, and set buttons for analog output channels
        for i, ao_channel in enumerate(self.ao_channels):
            ao_label = QLabel(f"Analog Output {ao_channel}")
            ao_upload_button = QPushButton("Upload")
            ao_upload_button.clicked.connect(partial(self.upload_ao_values, ao_channel))

            ao_period_label = QLabel("Period (s):")
            ao_period_textbox = QLineEdit("0.005")  # Default period of 0.005 seconds (200 Hz)
            ao_iterations_label = QLabel("Iterations:")
            ao_iterations_textbox = QLineEdit("1")  # Default iterations of 1

            ao_set_button = QPushButton("Set")
            ao_set_button.clicked.connect(partial(self.set_ao_settings, ao_channel))

            control_layout.addWidget(ao_label, i, 0)
            control_layout.addWidget(ao_upload_button, i, 1)
            control_layout.addWidget(ao_period_label, i, 2)
            control_layout.addWidget(ao_period_textbox, i, 3)
            control_layout.addWidget(ao_iterations_label, i, 4)
            control_layout.addWidget(ao_iterations_textbox, i, 5)
            control_layout.addWidget(ao_set_button, i, 6)

            self.ao_labels.append(ao_label)
            self.ao_period_textboxes.append(ao_period_textbox)
            self.ao_iterations_textboxes.append(ao_iterations_textbox)

        # Create labels, read buttons, terminal configuration, sampling rate, and voltage range input fields for analog input channels
        for i, ai_channel in enumerate(self.ai_channels):
            ai_label = QLabel(f"Analog Input {ai_channel}")
            ai_value_label = QLabel("0.0")

            ai_terminal_config = QComboBox()
            ai_terminal_config.addItems(self.terminal_configs)
            self.ai_terminal_configs.append(ai_terminal_config)

            ai_sampling_rate = QLineEdit("1000")
            self.ai_sampling_rates.append(ai_sampling_rate)

            ai_min_voltage = QLineEdit("-10")
            ai_max_voltage = QLineEdit("10")
            self.ai_min_voltages.append(ai_min_voltage)
            self.ai_max_voltages.append(ai_max_voltage)

            control_layout.addWidget(ai_label, i + len(self.ao_channels), 0)
            control_layout.addWidget(QLabel("Terminal Config:"), i + len(self.ao_channels), 2)
            control_layout.addWidget(ai_terminal_config, i + len(self.ao_channels), 3)
            control_layout.addWidget(QLabel("Min Voltage:"), i + len(self.ao_channels), 6)
            control_layout.addWidget(ai_min_voltage, i + len(self.ao_channels), 7)
            control_layout.addWidget(QLabel("Max Voltage:"), i + len(self.ao_channels), 8)
            control_layout.addWidget(ai_max_voltage, i + len(self.ao_channels), 9)

            self.ai_labels.append(ai_value_label)
        
        control_widget = QWidget()
        control_widget.setLayout(control_layout)

        scroll_area = QScrollArea()
        scroll_area.setWidget(control_widget)
        scroll_area.setWidgetResizable(True)
        scroll_area.setFixedHeight(400)

        main_layout.addWidget(scroll_area)

        # Create plot widgets for each analog output channel
        for i, ao_channel in enumerate(self.ao_channels):
            plot_widget_ao = pg.PlotWidget(title=f"Analog Output Wave {ao_channel}")
            plot_widget_ao.setLabel('left', 'Voltage', units='V')
            plot_widget_ao.setLabel('bottom', 'Time', units='s')
            plot_widget_ao.showGrid(x=True, y=True)
            plot_widget_ao.setYRange(-10, 10, padding=0)
            plot_widget_ao.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            plot_widget_ao.setFixedSize(300, 200)
            self.ao_plots[ao_channel] = plot_widget_ao
            plot_layout.addWidget(plot_widget_ao, i, 0)
            
            # Create separate preview plot widgets for each analog output channel
            preview_plot = pg.PlotWidget(title=f"Preview: {ao_channel}")
            preview_plot.setLabel('left', 'Voltage', units='V')
            preview_plot.setLabel('bottom', 'Sample')
            preview_plot.showGrid(x=True, y=True)
            preview_plot.setYRange(-10, 10, padding=0)
            preview_plot.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            preview_plot.setFixedSize(300, 200)
            self.ao_preview_plots[ao_channel] = preview_plot
            plot_layout.addWidget(preview_plot, i,1)

        for i, ai_channel in enumerate(self.ai_channels):
            plot_widget_ai = pg.PlotWidget(title=f"Analog Input Wave {ai_channel}")
            plot_widget_ai.setLabel('left', 'Voltage', units='V')
            plot_widget_ai.setLabel('bottom', 'Time', units='s')
            plot_widget_ai.showGrid(x=True, y=True)
            plot_widget_ai.setYRange(-10, 10, padding=0)
            plot_widget_ai.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            plot_widget_ai.setFixedSize(300, 200)
            self.ai_plots[ai_channel] = plot_widget_ai
            plot_layout.addWidget(plot_widget_ai,i,2)

        plot_widget = QWidget()
        plot_widget.setLayout(plot_layout)

        plot_scroll_area = QScrollArea()
        plot_scroll_area.setWidget(plot_widget)
        plot_scroll_area.setWidgetResizable(True)

        main_layout.addWidget(plot_scroll_area)
        self.setLayout(main_layout)
        
        self.setWindowTitle('DAQmx Analog IO Controller')
        self.showMaximized()
        # Add reset buttons for analog output graphs
        for i, ao_channel in enumerate(self.ao_channels):
            ao_reset_button = QPushButton("Reset")
            ao_reset_button.clicked.connect(partial(self.reset_ao_graph, ao_channel))
            control_layout.addWidget(ao_reset_button, i, 11)  # Change the column to 7

        # Add reset buttons for analog input graphs
        for i, ai_channel in enumerate(self.ai_channels):
            ai_reset_button = QPushButton("Reset")
            ai_reset_button.clicked.connect(partial(self.reset_ai_graph, ai_channel))
            control_layout.addWidget(ai_reset_button, i + len(self.ao_channels), 11)
        
        # Add read analog output button and digit display for each channel
        self.ao_read_labels = []
        self.ao_read_labels = []
        for i, ao_channel in enumerate(self.ao_channels):
            ao_read_label = QLabel("0.0")
            self.ao_read_labels.append(ao_read_label)
            control_layout.addWidget(ao_read_label, i, 7)  # Change the column to 7
        
    def set_ao_value(self, channel, textbox):
        try:
            value = float(textbox.text())
            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.write(value)
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error setting AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def sweep_ao_voltage(self, channel, start_voltage, end_voltage, voltage_step, iterations):
        try:
            start = float(start_voltage.text())
            end = float(end_voltage.text())
            step = float(voltage_step.text())
            num_iterations = int(iterations.text())

            if start >= end:
                raise ValueError("Start voltage must be less than end voltage.")
            if step <= 0:
                raise ValueError("Voltage step must be greater than zero.")
            if num_iterations <= 0:
                raise ValueError("Number of iterations must be greater than zero.")

            voltages = np.arange(start, end + step, step)
            waveform = np.tile(voltages, num_iterations)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.timing.cfg_samp_clk_timing(rate=1000, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                task.write(waveform, auto_start=True)

                index = self.ao_channels.index(channel)
                timestamps = np.linspace(0, len(waveform) / 1000, len(waveform))
                for i in range(len(waveform)):
                    self.ao_values[index].append(waveform[i])
                    self.ao_timestamps[index].append(timestamps[i])
                    self.ao_plots[channel].clear()
                    self.ao_plots[channel].plot(self.ao_timestamps[index], self.ao_values[index])
                    QApplication.processEvents()  # Process events to keep the UI responsive

                task.wait_until_done()
                task.stop()

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error sweeping AO voltage: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def set_ao_settings(self, channel):
        try:
            index = self.ao_channels.index(channel)
            period = float(self.ao_period_textboxes[index].text())
            iterations = int(self.ao_iterations_textboxes[index].text())

            if period <= 0:
                raise ValueError("Period must be greater than zero.")
            if iterations <= 0:
                raise ValueError("Iterations must be greater than zero.")

            if self.uploaded_data[channel] is None:
                raise ValueError("No data uploaded for this channel.")

            values = self.uploaded_data[channel]
            samples = len(values)
            t = np.linspace(0, period * iterations, samples * iterations)
            preview_waveform = np.tile(values, iterations)

            # Update the preview plot
            self.ao_preview_plots[channel].clear()
            self.ao_preview_plots[channel].plot(t, preview_waveform)
            self.ao_preview_plots[channel].setLabel('bottom', 'Time', units='s')

            QMessageBox.information(self, "Settings Applied", f"Settings applied for {channel}. Check the preview graph.")

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Error setting AO values: {str(e)}")
            
    def upload_ao_values(self, channel):
        try:
            file_dialog = QFileDialog()
            file_path, _ = file_dialog.getOpenFileName(self, "Open Text File", "", "Text Files (*.txt)")
            if file_path:
                worker = Worker(self._load_file, file_path, channel)
                worker.signals.result.connect(self.update_preview)
                worker.signals.error.connect(self.handle_error)
                self.threadpool.start(worker)
        except Exception as e:
            self.handle_error(("Error", e, ""))

    def _load_file(self, file_path, channel):
        with open(file_path, "r") as file:
            values = [float(line.strip()) for line in file.readlines()]
        return channel, values

    def update_preview(self, result):
        channel, values = result
        self.uploaded_data[channel] = values
        index = self.ao_channels.index(channel)
        
        # Update preview plot
        self.ao_preview_plots[channel].clear()
        self.ao_preview_plots[channel].plot(range(len(values)), values)
        
        # Enable the "Run" button
        self.run_buttons[index].setEnabled(True)
    
    def closeEvent(self, event):
        # Ensure all tasks are closed and resources are released
        for task in nidaqmx.system.System().tasks:
            try:
                task.close()
            except:
                pass
        event.accept()

    def run_ao_values(self, channel):
        if self.uploaded_data[channel] is None:
            QMessageBox.warning(self, "Warning", "No data uploaded for this channel.")
            return
        
        index = self.ao_channels.index(channel)
        values = self.uploaded_data[channel]
        period = float(self.ao_period_textboxes[index].text())
        iterations = int(self.ao_iterations_textboxes[index].text())

        sample_rate = len(values) / period
        samples_per_channel = len(values) * iterations

        waveform = np.tile(values, iterations)

        try:
            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO task
                ao_task.ao_channels.add_ao_voltage_chan(channel)
                ao_task.timing.cfg_samp_clk_timing(rate=sample_rate,
                                                sample_mode=AcquisitionType.FINITE,
                                                samps_per_chan=samples_per_channel)

                # Configure AI task
                ai_channel = f"Dev1/ai{channel[-1]}"
                terminal_config = getattr(TerminalConfiguration, self.ai_terminal_configs[index].currentText())
                min_val = float(self.ai_min_voltages[index].text())
                max_val = float(self.ai_max_voltages[index].text())
                
                ai_task.ai_channels.add_ai_voltage_chan(ai_channel,
                                                        terminal_config=terminal_config,
                                                        min_val=min_val,
                                                        max_val=max_val)
                ai_task.timing.cfg_samp_clk_timing(rate=sample_rate,
                                                source="ao/SampleClock",
                                                sample_mode=AcquisitionType.FINITE,
                                                samps_per_chan=samples_per_channel,
                                                active_edge=Edge.RISING)

                # Prepare the writer and reader
                ao_writer = AnalogMultiChannelWriter(ao_task.out_stream)
                ai_reader = AnalogMultiChannelReader(ai_task.in_stream)

                # Write data to AO buffer
                ao_writer.write_many_sample(waveform.reshape(1, -1))

                # Prepare buffer for AI data
                ai_data = np.zeros((1, samples_per_channel))

                # Start tasks
                ai_task.start()
                start_time = time.perf_counter()
                ao_task.start()

                # Read data
                ai_reader.read_many_sample(ai_data, number_of_samples_per_channel=samples_per_channel,
                                        timeout=period * iterations + 5.0)

                end_time = time.perf_counter()

                # Clip AI data to specified range
                ai_data = np.clip(ai_data, min_val, max_val)

            # Generate timestamps
            timestamps = np.linspace(0, period * iterations, samples_per_channel)

            # Update plots
            self.update_plot(channel, timestamps, waveform, timestamps, ai_data[0])

            # Calculate and display timing information
            expected_duration = period * iterations
            actual_duration = end_time - start_time
            timing_error = abs(actual_duration - expected_duration)

            info_message = (f"Waveform output and input completed for channel {channel}\n"
                            f"Expected duration: {expected_duration:.6f}s\n"
                            f"Actual duration: {actual_duration:.6f}s\n"
                            f"Timing error: {timing_error:.6f}s")
            
            QMessageBox.information(self, "Success", info_message)

        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error during AO/AI operation: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")
        
    def update_plots(self, channel, ao_timestamps, ao_values, ai_timestamps, ai_values):
        try:
            # Update AO plot
            self.ao_plots[channel].clear()
            self.ao_plots[channel].plot(ao_timestamps, ao_values)
            
            # Update AI plot
            ai_channel = f"Dev1/ai{self.ao_channels.index(channel)}"
            self.ai_plots[ai_channel].clear()
            self.ai_plots[ai_channel].plot(ai_timestamps, ai_values)
            
            QApplication.processEvents()
        except Exception as e:
            print(f"Error updating plots: {str(e)}")
    
    def _output_and_read_values(self, channel, waveform, timestamps, total_duration):
        try:
            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO task
                ao_task.ao_channels.add_ao_voltage_chan(channel)
                ao_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                
                # Configure AI task
                ai_channel = f"Dev1/ai{self.ao_channels.index(channel)}"
                ai_task.ai_channels.add_ai_voltage_chan(ai_channel)
                ai_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                
                # Set up synchronization
                ai_task.triggers.start_trigger.cfg_dig_edge_start_trig(f"/{channel}/StartTrigger")
                
                # Write AO data
                ao_writer = AnalogMultiChannelWriter(ao_task.out_stream)
                ao_writer.write_many_sample(np.array([waveform]))
                
                # Prepare AI reader
                ai_reader = AnalogMultiChannelReader(ai_task.in_stream)
                ai_data = np.zeros((1, len(waveform)))
                
                # Start tasks
                ai_task.start()
                ao_task.start()
                
                # Read and update plots
                for i in range(0, len(waveform), 1000):
                    if ao_task.is_task_done():
                        break
                    progress = int((i / len(waveform)) * 100)
                    self.signals.progress.emit(progress)
                    
                    ai_reader.read_many_sample(ai_data[:, i:i+1000], number_of_samples_per_channel=min(1000, len(waveform)-i))
                    self.signals.result.emit((timestamps[:i+1000], waveform[:i+1000], ai_data[0, :i+1000]))
                    time.sleep(0.1)  # Adjust this value to control update frequency
                
                ao_task.wait_until_done(timeout=total_duration + 5.0)
                ai_task.wait_until_done(timeout=total_duration + 5.0)
        except Exception as e:
            self.signals.error.emit(str(e))

    def update_ao_plot(self, channel, timestamps, values):
        self.ao_plots[channel].clear()
        self.ao_plots[channel].plot(timestamps * 1000, values)  # Convert timestamps to milliseconds for display
        self.ao_plots[channel].setLabel('bottom', 'Time', units='ms')
        self.ao_plots[channel].setXRange(0, timestamps[-1] * 1000)  # Ensure full range is visible
        QApplication.processEvents()
        
    def handle_thread_error(self, error_message):
        QMessageBox.critical(self, "Error", f"An error occurred: {error_message}")

    def output_finished(self, channel):
        QMessageBox.information(self, "Success", f"Waveform output completed for channel {channel}")
        
    def _output_ao_values(self, channel):
        index = self.ao_channels.index(channel)
        values = self.uploaded_data[channel]
        period = float(self.ao_period_textboxes[index].text())
        iterations = int(self.ao_iterations_textboxes[index].text())

        waveform = np.tile(values, iterations)
        total_duration = period * iterations * len(values)

        with nidaqmx.Task() as task:
            task.ao_channels.add_ao_voltage_chan(channel)
            task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
            task.write(waveform, auto_start=True)
            task.wait_until_done(timeout=total_duration + 5.0)

    def output_finished(self, channel):
        logging.debug(f"Output finished for channel {channel}")
        QMessageBox.information(self, "Success", f"Output completed for channel {channel}")

    def handle_error(self, error_info):
        QMessageBox.critical(self, "Error", str(error_info[1]))
                
    def output_ao_values(self, channel):
        try:
            index = self.ao_channels.index(channel)
            file_path = self.ao_file_paths[index]
            period = float(self.ao_period_textboxes[index].text())
            iterations = int(self.ao_iterations_textboxes[index].text())

            if file_path is None:
                raise ValueError("No file uploaded for this channel.")

            with open(file_path, "r") as file:
                values = [float(line.strip()) for line in file.readlines()]

            # Prepare the waveform data
            waveform = np.tile(values, iterations)
            total_duration = period * iterations * len(values)

            self.progress_bar.setVisible(True)
            self.progress_bar.setValue(0)

            # Create tasks for both output and input
            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO task
                ao_task.ao_channels.add_ao_voltage_chan(channel)
                ao_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))

                # Configure AI task
                ai_channel = f"Dev1/ai{index}"
                ai_task.ai_channels.add_ai_voltage_chan(
                    ai_channel,
                    terminal_config=getattr(TerminalConfiguration, self.ai_terminal_configs[index].currentText()),
                    min_val=float(self.ai_min_voltages[index].text()),
                    max_val=float(self.ai_max_voltages[index].text())
                )
                ai_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))

                data_thread = DataProcessingThread(channel, waveform, total_duration, ai_task, ao_task)
                data_thread.update_progress.connect(self.progress_bar.setValue)
                data_thread.update_plot.connect(self.update_plot)
                data_thread.finished.connect(lambda: self.progress_bar.setVisible(False))
                data_thread.start()

                # Wait for the thread to finish
                data_thread.wait()

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error outputting AO values: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")
        finally:
            self.progress_bar.setVisible(False)
    
    def handle_thread_error(self, error_message):
        QMessageBox.critical
    
    def cleanup_tasks(self, ao_task, ai_task):
        try:
            ao_task.close()
            ai_task.close()
        except Exception as e:
            print(f"Error during task cleanup: {str(e)}")
        finally:
            self.progress_bar.setVisible(False)
            
    def update_progress(self, value):
        self.progress_bar.setValue(value)

    
    def update_plot(self, channel, ao_timestamps, ao_values, ai_timestamps, ai_values):
        
        index = self.ao_channels.index(channel)
        
        # Update AO plot
        self.ao_values[index] = ao_values
        self.ao_timestamps[index] = ao_timestamps
        self.ao_plots[channel].clear()
        self.ao_plots[channel].plot(ao_timestamps * 1000, ao_values)  # Convert to milliseconds
        self.ao_plots[channel].setLabel('bottom', 'Time', units='ms')
        self.ao_plots[channel].setXRange(0, ao_timestamps[-1] * 1000)
        
        # Update AI plot
        ai_channel = f"Dev1/ai{index}"
        self.ai_values[index] = ai_values
        self.ai_timestamps[index] = ai_timestamps
        self.ai_plots[ai_channel].clear()
        self.ai_plots[ai_channel].plot(ai_timestamps * 1000, ai_values)  # Convert to milliseconds
        self.ai_plots[ai_channel].setLabel('bottom', 'Time', units='ms')
        self.ai_plots[ai_channel].setXRange(0, ai_timestamps[-1] * 1000)
        
        QApplication.processEvents()

                            
    def reset_ao_graph(self, channel):
        index = self.ao_channels.index(channel)
        self.ao_values[index] = np.array([])
        self.ao_timestamps[index] = np.array([])
        self.ao_plots[channel].clear()

    def reset_ai_graph(self, channel):
        index = self.ai_channels.index(channel)
        self.ai_values[index] = np.array([])
        self.ai_timestamps[index] = np.array([])
        self.ai_plots[channel].clear()
    
    def read_ao_value(self, channel, index):
        try:
            with nidaqmx.Task() as task:
                task.ai_channels.add_ai_voltage_chan(channel)  # Change to ai_channels
                value = task.read()
                self.ao_read_labels[index].setText(f"{value[0]:.2f}")
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error reading AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = DAQmxController()
    sys.exit(app.exec_())

In [ ]:
#Multi channel

import sys
import numpy as np
import time
from functools import partial
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QHBoxLayout, QGridLayout, 
    QScrollArea, QComboBox, QLineEdit, QFileDialog, QMessageBox, QProgressBar, QProgressDialog
)
from PyQt5.QtCore import Qt, QThread, pyqtSignal, QRunnable, QThreadPool, QObject
import pyqtgraph as pg
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, AcquisitionType, Edge
from nidaqmx.stream_writers import AnalogMultiChannelWriter
from nidaqmx.stream_readers import AnalogMultiChannelReader


class WorkerSignals(QObject):
    finished = pyqtSignal()
    error = pyqtSignal(tuple)
    result = pyqtSignal(object)

class Worker(QRunnable):
    def __init__(self, fn, *args, **kwargs):
        super().__init__()
        self.fn = fn
        self.args = args
        self.kwargs = kwargs
        self.signals = WorkerSignals()

    def run(self):
        try:
            result = self.fn(*self.args, **self.kwargs)
        except:
            exctype, value = sys.exc_info()[:2]
            self.signals.error.emit((exctype, value, traceback.format_exc()))
        else:
            self.signals.result.emit(result)
        finally:
            self.signals.finished.emit()

class OutputThread(QThread):
    update_plot = pyqtSignal(str, np.ndarray, np.ndarray)
    error_occurred = pyqtSignal(str)
    finished = pyqtSignal()

    def __init__(self, channel, values, period, iterations, sample_rate):
        super().__init__()
        self.channel = channel
        self.values = values
        self.period = period
        self.iterations = iterations
        self.sample_rate = sample_rate

    def run(self):
        try:
            single_period_samples = len(self.values)
            total_samples = single_period_samples * self.iterations
            waveform = np.tile(self.values, self.iterations)
            
            # Calculate total duration based on period and iterations
            total_duration = self.period * self.iterations
            timestamps = np.linspace(0, total_duration, total_samples)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(self.channel)
                task.timing.cfg_samp_clk_timing(rate=self.sample_rate, sample_mode=AcquisitionType.FINITE, samps_per_chan=total_samples)
                
                task.write(waveform, auto_start=True)
                
                self.update_plot.emit(self.channel, timestamps, waveform)
                
                task.wait_until_done(timeout=total_duration + 5.0)
            
            self.finished.emit()
        except Exception as e:
            self.error_occurred.emit(str(e))
            
class DataProcessingThread(QThread):
    update_progress = pyqtSignal(int)
    update_plot = pyqtSignal(str, np.ndarray, np.ndarray, np.ndarray, np.ndarray)
    error_occurred = pyqtSignal(str)
    
    def __init__(self, channel, waveform, total_duration, ai_task, ao_task):
        super().__init__()
        self.channel = channel
        self.waveform = waveform
        self.total_duration = total_duration
        self.stop_event = threading.Event()

    def run(self):
        try:
            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO task
                ao_task.ao_channels.add_ao_voltage_chan(self.channel)
                ao_task.timing.cfg_samp_clk_timing(rate=len(self.waveform)/self.total_duration, 
                                                   sample_mode=AcquisitionType.FINITE, 
                                                   samps_per_chan=len(self.waveform))

                # Configure AI task
                ai_channel = f"Dev1/ai{self.channel[-1]}"  # Assumes channel format like "Dev1/ao0"
                ai_task.ai_channels.add_ai_voltage_chan(ai_channel)
                ai_task.timing.cfg_samp_clk_timing(rate=len(self.waveform)/self.total_duration, 
                                                   sample_mode=AcquisitionType.FINITE, 
                                                   samps_per_chan=len(self.waveform))

                # Set up synchronization
                ai_task.triggers.start_trigger.cfg_dig_edge_start_trig(f"/{self.channel}/StartTrigger")

                writer = AnalogMultiChannelWriter(ao_task.out_stream)
                reader = AnalogMultiChannelReader(ai_task.in_stream)

                writer.write_many_sample(np.array([self.waveform]))
                
                ao_task.start()
                ai_task.start()

                ao_data = self.waveform
                ai_data = np.zeros((1, len(self.waveform)))
                timestamps = np.linspace(0, self.total_duration, len(self.waveform))

                batch_size = 1000
                for i in range(0, len(self.waveform), batch_size):
                    if self.stop_event.is_set():
                        break
                    end = min(i + batch_size, len(self.waveform))
                    reader.read_many_sample(
                        ai_data[:, i:end], 
                        number_of_samples_per_channel=end - i,
                        timeout=5.0
                    )
                    progress = int((i / len(self.waveform)) * 100)
                    self.update_progress.emit(progress)
                    
                    self.update_plot.emit(self.channel, timestamps[:end], ao_data[:end], timestamps[:end], ai_data[0, :end])

                ao_task.wait_until_done(timeout=self.total_duration + 5.0)
                ai_task.wait_until_done(timeout=self.total_duration + 5.0)
                
                self.update_plot.emit(self.channel, timestamps, ao_data, timestamps, ai_data[0])
                self.update_progress.emit(100)

        except Exception as e:
            self.error_occurred.emit(str(e))
        finally:
            self.stop_event.set()

    def stop(self):
        self.stop_event.set()

class DAQmxController(QWidget):
    def __init__(self):
        super().__init__()
        self.threadpool = QThreadPool()

        self.ao_channels = ["Dev1/ao0", "Dev1/ao1", "Dev1/ao2", "Dev1/ao3"]
        self.ai_channels = ["Dev1/ai0", "Dev1/ai1", "Dev1/ai2", "Dev1/ai3"]
        self.terminal_configs = ["RSE", "Differential", "Pseudodifferential"]

        self.ao_values = [np.array([]) for _ in range(len(self.ao_channels))]
        self.ai_values = [np.array([]) for _ in range(len(self.ai_channels))]
        self.ao_timestamps = [np.array([]) for _ in range(len(self.ao_channels))]
        self.ai_timestamps = [np.array([]) for _ in range(len(self.ai_channels))]
        
        self.ao_period_textboxes = []
        self.ao_iterations_textboxes = []
        self.ao_file_paths = [None] * len(self.ao_channels)
        self.progress_bar = QProgressBar(self)
        self.progress_bar.setVisible(False)
        self.uploaded_data = None
        self.run_buttons = []
        self.ao_preview_plots = {}  # New dictionary for preview plots
        
        self.initUI()

    def initUI(self):
        main_layout = QVBoxLayout()
        control_layout = QGridLayout()
        control_layout.setHorizontalSpacing(2)
        control_layout.setVerticalSpacing(2)
        
        upload_button = QPushButton("Upload Multichannel Data")
        upload_button.clicked.connect(self.upload_multichannel_data)
        control_layout.addWidget(upload_button, 0, 1)

        # Create a grid layout for the plots
        plot_layout = QGridLayout()
        plot_layout.setHorizontalSpacing(5)
        plot_layout.setVerticalSpacing(5)
        
        self.ao_labels = []
        self.ai_labels = []
        self.ao_plots = {}
        self.ai_plots = {}
        self.ai_terminal_configs = []
        self.ai_sampling_rates = []
        self.ai_min_voltages = []
        self.ai_max_voltages = []

        # Add a single "Run All Channels" button
        self.run_button = QPushButton("Run All Channels")
        self.run_button.clicked.connect(self.run_ao_values)
        self.run_button.setEnabled(False)
        control_layout.addWidget(self.run_button, 0, 9)  # Adjust the row and column as needed
            
        # Create textboxes, labels, upload buttons, period, iterations fields, and set buttons for analog output channels
        for i, ao_channel in enumerate(self.ao_channels):
            ao_label = QLabel(f"Analog Output {ao_channel}")

            ao_period_label = QLabel("Period (s):")
            ao_period_textbox = QLineEdit("0.005")  # Default period of 0.005 seconds (200 Hz)
            ao_iterations_label = QLabel("Iterations:")
            ao_iterations_textbox = QLineEdit("1")  # Default iterations of 1

            ao_set_button = QPushButton("Set")
            ao_set_button.clicked.connect(partial(self.set_ao_settings, ao_channel))

            control_layout.addWidget(ao_label, i, 0)
            control_layout.addWidget(ao_period_label, i, 2)
            control_layout.addWidget(ao_period_textbox, i, 3)
            control_layout.addWidget(ao_iterations_label, i, 4)
            control_layout.addWidget(ao_iterations_textbox, i, 5)
            control_layout.addWidget(ao_set_button, i, 6)
            
            self.ao_labels.append(ao_label)
            self.ao_period_textboxes.append(ao_period_textbox)
            self.ao_iterations_textboxes.append(ao_iterations_textbox)

        # Create labels, read buttons, terminal configuration, sampling rate, and voltage range input fields for analog input channels
        for i, ai_channel in enumerate(self.ai_channels):
            ai_label = QLabel(f"Analog Input {ai_channel}")
            ai_value_label = QLabel("0.0")

            ai_terminal_config = QComboBox()
            ai_terminal_config.addItems(self.terminal_configs)
            self.ai_terminal_configs.append(ai_terminal_config)

            ai_sampling_rate = QLineEdit("1000")
            self.ai_sampling_rates.append(ai_sampling_rate)

            ai_min_voltage = QLineEdit("-10")
            ai_max_voltage = QLineEdit("10")
            self.ai_min_voltages.append(ai_min_voltage)
            self.ai_max_voltages.append(ai_max_voltage)

            control_layout.addWidget(ai_label, i + len(self.ao_channels), 0)
            control_layout.addWidget(QLabel("Terminal Config:"), i + len(self.ao_channels), 2)
            control_layout.addWidget(ai_terminal_config, i + len(self.ao_channels), 3)
            control_layout.addWidget(QLabel("Min Voltage:"), i + len(self.ao_channels), 6)
            control_layout.addWidget(ai_min_voltage, i + len(self.ao_channels), 7)
            control_layout.addWidget(QLabel("Max Voltage:"), i + len(self.ao_channels), 8)
            control_layout.addWidget(ai_max_voltage, i + len(self.ao_channels), 9)

            self.ai_labels.append(ai_value_label)
        
        control_widget = QWidget()
        control_widget.setLayout(control_layout)

        scroll_area = QScrollArea()
        scroll_area.setWidget(control_widget)
        scroll_area.setWidgetResizable(True)
        scroll_area.setFixedHeight(400)

        main_layout.addWidget(scroll_area)

        # Create plot widgets for each analog output channel
        for i, ao_channel in enumerate(self.ao_channels):
            plot_widget_ao = pg.PlotWidget(title=f"Analog Output Wave {ao_channel}")
            plot_widget_ao.setLabel('left', 'Voltage', units='V')
            plot_widget_ao.setLabel('bottom', 'Time', units='s')
            plot_widget_ao.showGrid(x=True, y=True)
            plot_widget_ao.setYRange(-10, 10, padding=0)
            plot_widget_ao.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            plot_widget_ao.setFixedSize(300, 200)
            self.ao_plots[ao_channel] = plot_widget_ao
            plot_layout.addWidget(plot_widget_ao, i, 0)
            
            # Create separate preview plot widgets for each analog output channel
            preview_plot = pg.PlotWidget(title=f"Preview: {ao_channel}")
            preview_plot.setLabel('left', 'Voltage', units='V')
            preview_plot.setLabel('bottom', 'Sample')
            preview_plot.showGrid(x=True, y=True)
            preview_plot.setYRange(-10, 10, padding=0)
            preview_plot.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            preview_plot.setFixedSize(300, 200)
            self.ao_preview_plots[ao_channel] = preview_plot
            plot_layout.addWidget(preview_plot, i,1)

        for i, ai_channel in enumerate(self.ai_channels):
            plot_widget_ai = pg.PlotWidget(title=f"Analog Input Wave {ai_channel}")
            plot_widget_ai.setLabel('left', 'Voltage', units='V')
            plot_widget_ai.setLabel('bottom', 'Time', units='s')
            plot_widget_ai.showGrid(x=True, y=True)
            plot_widget_ai.setYRange(-10, 10, padding=0)
            plot_widget_ai.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            plot_widget_ai.setFixedSize(300, 200)
            self.ai_plots[ai_channel] = plot_widget_ai
            plot_layout.addWidget(plot_widget_ai,i,2)

        plot_widget = QWidget()
        plot_widget.setLayout(plot_layout)

        plot_scroll_area = QScrollArea()
        plot_scroll_area.setWidget(plot_widget)
        plot_scroll_area.setWidgetResizable(True)

        main_layout.addWidget(plot_scroll_area)
        self.setLayout(main_layout)
        
        self.setWindowTitle('DAQmx Analog IO Controller')
        self.showMaximized()
        # Add reset buttons for analog output graphs
        for i, ao_channel in enumerate(self.ao_channels):
            ao_reset_button = QPushButton("Reset")
            ao_reset_button.clicked.connect(partial(self.reset_ao_graph, ao_channel))
            control_layout.addWidget(ao_reset_button, i, 11)  

        # Add reset buttons for analog input graphs
        for i, ai_channel in enumerate(self.ai_channels):
            ai_reset_button = QPushButton("Reset")
            ai_reset_button.clicked.connect(partial(self.reset_ai_graph, ai_channel))
            control_layout.addWidget(ai_reset_button, i + len(self.ao_channels), 11)
        
        # Add read analog output button and digit display for each channel
        self.ao_read_labels = []
        self.ao_read_labels = []
        for i, ao_channel in enumerate(self.ao_channels):
            ao_read_label = QLabel("0.0")
            self.ao_read_labels.append(ao_read_label)
            control_layout.addWidget(ao_read_label, i, 7)  # Change the column to 7

    def set_ao_value(self, channel, textbox):
        try:
            value = float(textbox.text())
            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.write(value)
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error setting AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def sweep_ao_voltage(self, channel, start_voltage, end_voltage, voltage_step, iterations):
        try:
            start = float(start_voltage.text())
            end = float(end_voltage.text())
            step = float(voltage_step.text())
            num_iterations = int(iterations.text())

            if start >= end:
                raise ValueError("Start voltage must be less than end voltage.")
            if step <= 0:
                raise ValueError("Voltage step must be greater than zero.")
            if num_iterations <= 0:
                raise ValueError("Number of iterations must be greater than zero.")

            voltages = np.arange(start, end + step, step)
            waveform = np.tile(voltages, num_iterations)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.timing.cfg_samp_clk_timing(rate=1000, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                task.write(waveform, auto_start=True)

                index = self.ao_channels.index(channel)
                timestamps = np.linspace(0, len(waveform) / 1000, len(waveform))
                for i in range(len(waveform)):
                    self.ao_values[index].append(waveform[i])
                    self.ao_timestamps[index].append(timestamps[i])
                    self.ao_plots[channel].clear()
                    self.ao_plots[channel].plot(self.ao_timestamps[index], self.ao_values[index])
                    QApplication.processEvents()  # Process events to keep the UI responsive

                task.wait_until_done()
                task.stop()

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error sweeping AO voltage: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def set_ao_settings(self, channel):
        try:
            period = float(self.ao_period_textboxes[0].text())
            iterations = int(self.ao_iterations_textboxes[0].text())

            if period <= 0:
                raise ValueError("Period must be greater than zero.")
            if iterations <= 0:
                raise ValueError("Iterations must be greater than zero.")

            if self.uploaded_data is None:
                raise ValueError("No data uploaded.")

            samples = self.uploaded_data.shape[1]
            t = np.linspace(0, period * iterations, samples * iterations)
            preview_waveform = np.tile(self.uploaded_data, (1, iterations))

            # Update the preview plots
            for i, channel in enumerate(self.ao_channels):
                if i < self.uploaded_data.shape[0]:
                    self.ao_preview_plots[channel].clear()
                    self.ao_preview_plots[channel].plot(t, preview_waveform[i])
                    self.ao_preview_plots[channel].setLabel('bottom', 'Time', units='s')

            QMessageBox.information(self, "Settings Applied", "Settings applied for all channels. Check the preview graphs.")

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Error setting AO values: {str(e)}")

    def _load_file(self, file_path, channel):
        with open(file_path, "r") as file:
            values = [float(line.strip()) for line in file.readlines()]
        return channel, values


    
    def closeEvent(self, event):
        # Ensure all tasks are closed and resources are released
        for task in nidaqmx.system.System().tasks:
            try:
                task.close()
            except:
                pass
        event.accept()

    def run_ao_values(self):
        if self.uploaded_data is None:
            QMessageBox.warning(self, "Warning", "No data uploaded.")
            return
        
        try:
            period = float(self.ao_period_textboxes[0].text())
            iterations = int(self.ao_iterations_textboxes[0].text())

            sample_rate = self.uploaded_data.shape[1] / period
            samples_per_channel = self.uploaded_data.shape[1] * iterations

            # Ensure the waveform is contiguous in memory
            waveform = np.ascontiguousarray(np.tile(self.uploaded_data, (1, iterations)))

            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO and AI tasks for all channels
                for i, (ao_channel, ai_channel) in enumerate(zip(self.ao_channels, self.ai_channels)):
                    ao_task.ao_channels.add_ao_voltage_chan(ao_channel)
                    terminal_config = getattr(TerminalConfiguration, self.ai_terminal_configs[i].currentText())
                    min_val = float(self.ai_min_voltages[i].text())
                    max_val = float(self.ai_max_voltages[i].text())
                    ai_task.ai_channels.add_ai_voltage_chan(ai_channel, terminal_config=terminal_config, min_val=min_val, max_val=max_val)

                ao_task.timing.cfg_samp_clk_timing(rate=sample_rate, sample_mode=AcquisitionType.FINITE, samps_per_chan=samples_per_channel)
                ai_task.timing.cfg_samp_clk_timing(rate=sample_rate, source="ao/SampleClock", sample_mode=AcquisitionType.FINITE, samps_per_chan=samples_per_channel)

                ao_writer = AnalogMultiChannelWriter(ao_task.out_stream)
                ai_reader = AnalogMultiChannelReader(ai_task.in_stream)

                ao_writer.write_many_sample(waveform)

                ai_data = np.zeros((len(self.ai_channels), samples_per_channel), dtype=np.float64)

                ai_task.start()
                start_time = time.perf_counter()
                ao_task.start()

                ai_reader.read_many_sample(ai_data, number_of_samples_per_channel=samples_per_channel, timeout=period * iterations + 5.0)

                end_time = time.perf_counter()

            timestamps = np.linspace(0, period * iterations, samples_per_channel)

            for i, (ao_channel, ai_channel) in enumerate(zip(self.ao_channels, self.ai_channels)):
                self.update_plot(ao_channel, timestamps, waveform[i], timestamps, ai_data[i])

            # Calculate and display timing information
            expected_duration = period * iterations
            actual_duration = end_time - start_time
            timing_error = abs(actual_duration - expected_duration)

            info_message = (f"Multichannel waveform output and input completed\n"
                            f"Expected duration: {expected_duration:.6f}s\n"
                            f"Actual duration: {actual_duration:.6f}s\n"
                            f"Timing error: {timing_error:.6f}s")
            
            QMessageBox.information(self, "Success", info_message)

        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error during AO/AI operation: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")
        
    def update_plots(self, channel, ao_timestamps, ao_values, ai_timestamps, ai_values):
        try:
            # Update AO plot
            self.ao_plots[channel].clear()
            self.ao_plots[channel].plot(ao_timestamps, ao_values)
            
            # Update AI plot
            ai_channel = f"Dev1/ai{self.ao_channels.index(channel)}"
            self.ai_plots[ai_channel].clear()
            self.ai_plots[ai_channel].plot(ai_timestamps, ai_values)
            
            QApplication.processEvents()
        except Exception as e:
            print(f"Error updating plots: {str(e)}")
    
    def _output_and_read_values(self, channel, waveform, timestamps, total_duration):
        try:
            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO task
                ao_task.ao_channels.add_ao_voltage_chan(channel)
                ao_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                
                # Configure AI task
                ai_channel = f"Dev1/ai{self.ao_channels.index(channel)}"
                ai_task.ai_channels.add_ai_voltage_chan(ai_channel)
                ai_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                
                # Set up synchronization
                ai_task.triggers.start_trigger.cfg_dig_edge_start_trig(f"/{channel}/StartTrigger")
                
                # Write AO data
                ao_writer = AnalogMultiChannelWriter(ao_task.out_stream)
                ao_writer.write_many_sample(np.array([waveform]))
                
                # Prepare AI reader
                ai_reader = AnalogMultiChannelReader(ai_task.in_stream)
                ai_data = np.zeros((1, len(waveform)))
                
                # Start tasks
                ai_task.start()
                ao_task.start()
                
                # Read and update plots
                for i in range(0, len(waveform), 1000):
                    if ao_task.is_task_done():
                        break
                    progress = int((i / len(waveform)) * 100)
                    self.signals.progress.emit(progress)
                    
                    ai_reader.read_many_sample(ai_data[:, i:i+1000], number_of_samples_per_channel=min(1000, len(waveform)-i))
                    self.signals.result.emit((timestamps[:i+1000], waveform[:i+1000], ai_data[0, :i+1000]))
                    time.sleep(0.1)  # Adjust this value to control update frequency
                
                ao_task.wait_until_done(timeout=total_duration + 5.0)
                ai_task.wait_until_done(timeout=total_duration + 5.0)
        except Exception as e:
            self.signals.error.emit(str(e))

    def update_ao_plot(self, channel, timestamps, values):
        self.ao_plots[channel].clear()
        self.ao_plots[channel].plot(timestamps * 1000, values)  # Convert timestamps to milliseconds for display
        self.ao_plots[channel].setLabel('bottom', 'Time', units='ms')
        self.ao_plots[channel].setXRange(0, timestamps[-1] * 1000)  # Ensure full range is visible
        QApplication.processEvents()
        
    def handle_thread_error(self, error_message):
        QMessageBox.critical(self, "Error", f"An error occurred: {error_message}")

    def output_finished(self, channel):
        QMessageBox.information(self, "Success", f"Waveform output completed for channel {channel}")
        
    def _output_ao_values(self, channel):
        index = self.ao_channels.index(channel)
        values = self.uploaded_data[channel]
        period = float(self.ao_period_textboxes[index].text())
        iterations = int(self.ao_iterations_textboxes[index].text())

        waveform = np.tile(values, iterations)
        total_duration = period * iterations * len(values)

        with nidaqmx.Task() as task:
            task.ao_channels.add_ao_voltage_chan(channel)
            task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
            task.write(waveform, auto_start=True)
            task.wait_until_done(timeout=total_duration + 5.0)

    def output_finished(self, channel):
        logging.debug(f"Output finished for channel {channel}")
        QMessageBox.information(self, "Success", f"Output completed for channel {channel}")

    def handle_error(self, error_info):
        QMessageBox.critical(self, "Error", str(error_info[1]))
                
    def output_ao_values(self, channel):
        try:
            index = self.ao_channels.index(channel)
            file_path = self.ao_file_paths[index]
            period = float(self.ao_period_textboxes[index].text())
            iterations = int(self.ao_iterations_textboxes[index].text())

            if file_path is None:
                raise ValueError("No file uploaded for this channel.")

            with open(file_path, "r") as file:
                values = [float(line.strip()) for line in file.readlines()]

            # Prepare the waveform data
            waveform = np.tile(values, iterations)
            total_duration = period * iterations * len(values)

            self.progress_bar.setVisible(True)
            self.progress_bar.setValue(0)

            # Create tasks for both output and input
            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO task
                ao_task.ao_channels.add_ao_voltage_chan(channel)
                ao_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))

                # Configure AI task
                ai_channel = f"Dev1/ai{index}"
                ai_task.ai_channels.add_ai_voltage_chan(
                    ai_channel,
                    terminal_config=getattr(TerminalConfiguration, self.ai_terminal_configs[index].currentText()),
                    min_val=float(self.ai_min_voltages[index].text()),
                    max_val=float(self.ai_max_voltages[index].text())
                )
                ai_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))

                data_thread = DataProcessingThread(channel, waveform, total_duration, ai_task, ao_task)
                data_thread.update_progress.connect(self.progress_bar.setValue)
                data_thread.update_plot.connect(self.update_plot)
                data_thread.finished.connect(lambda: self.progress_bar.setVisible(False))
                data_thread.start()

                # Wait for the thread to finish
                data_thread.wait()

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error outputting AO values: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")
        finally:
            self.progress_bar.setVisible(False)
    
    def handle_thread_error(self, error_message):
        QMessageBox.critical
    
    def cleanup_tasks(self, ao_task, ai_task):
        try:
            ao_task.close()
            ai_task.close()
        except Exception as e:
            print(f"Error during task cleanup: {str(e)}")
        finally:
            self.progress_bar.setVisible(False)
            
    def update_progress(self, value):
        self.progress_bar.setValue(value)

    
    def update_plot(self, channel, ao_timestamps, ao_values, ai_timestamps, ai_values):
        
        index = self.ao_channels.index(channel)
        
        # Update AO plot
        self.ao_values[index] = ao_values
        self.ao_timestamps[index] = ao_timestamps
        self.ao_plots[channel].clear()
        self.ao_plots[channel].plot(ao_timestamps * 1000, ao_values)  # Convert to milliseconds
        self.ao_plots[channel].setLabel('bottom', 'Time', units='ms')
        self.ao_plots[channel].setXRange(0, ao_timestamps[-1] * 1000)
        
        # Update AI plot
        ai_channel = f"Dev1/ai{index}"
        self.ai_values[index] = ai_values
        self.ai_timestamps[index] = ai_timestamps
        self.ai_plots[ai_channel].clear()
        self.ai_plots[ai_channel].plot(ai_timestamps * 1000, ai_values)  # Convert to milliseconds
        self.ai_plots[ai_channel].setLabel('bottom', 'Time', units='ms')
        self.ai_plots[ai_channel].setXRange(0, ai_timestamps[-1] * 1000)
        
        QApplication.processEvents()

                            
    def reset_ao_graph(self, channel):
        index = self.ao_channels.index(channel)
        self.ao_values[index] = np.array([])
        self.ao_timestamps[index] = np.array([])
        self.ao_plots[channel].clear()

    def reset_ai_graph(self, channel):
        index = self.ai_channels.index(channel)
        self.ai_values[index] = np.array([])
        self.ai_timestamps[index] = np.array([])
        self.ai_plots[channel].clear()
    
    def read_ao_value(self, channel, index):
        try:
            with nidaqmx.Task() as task:
                task.ai_channels.add_ai_voltage_chan(channel)  # Change to ai_channels
                value = task.read()
                self.ao_read_labels[index].setText(f"{value[0]:.2f}")
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error reading AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")
    
    def upload_multichannel_data(self):
        try:
            file_dialog = QFileDialog()
            file_path, _ = file_dialog.getOpenFileName(self, "Open Multichannel Data File", "", "Text Files (*.txt)")
            if file_path:
                worker = Worker(self._load_multichannel_file, file_path)
                worker.signals.result.connect(self.update_multichannel_preview)
                worker.signals.error.connect(self.handle_error)
                self.threadpool.start(worker)
        except Exception as e:
            self.handle_error(("Error", e, ""))

    def _load_multichannel_file(self, file_path):
        data = np.loadtxt(file_path, delimiter='\t')
        return data.T  # Transpose the data so each row represents a channel

    def update_multichannel_preview(self, data):
        self.uploaded_data = data
        for i, channel in enumerate(self.ao_channels):
            if i < data.shape[0]:
                self.ao_preview_plots[channel].clear()
                self.ao_preview_plots[channel].plot(data[i])
        self.run_button.setEnabled(True)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = DAQmxController()
    sys.exit(app.exec_())

In [ ]:
# single channel sampling (untied)
# analog output and input tasks untied from the main thread. The 2 ports act independently.
# Cant perform only 1 analog input port can be activated at a single time.
# The multichannel not working.

import sys
import numpy as np
import time
from functools import partial
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QHBoxLayout, QGridLayout, 
    QScrollArea, QComboBox, QLineEdit, QFileDialog, QMessageBox, QProgressDialog, QProgressBar
)
from PyQt5.QtCore import Qt, QThread, pyqtSignal, QRunnable, QThreadPool, QObject
import pyqtgraph as pg
import nidaqmx
from nidaqmx.constants import AcquisitionType, Edge, TerminalConfiguration
from nidaqmx.stream_writers import AnalogMultiChannelWriter
from PyQt5.QtCore import QTimer
import threading

class AnalogInputTask(QThread):
    update_plot = pyqtSignal(np.ndarray, np.ndarray)

    def __init__(self, channel, sample_rate, buffer_size, config='RSE'):
        super().__init__()
        self.channel = channel
        self.sample_rate = sample_rate
        self.buffer_size = buffer_size
        self.config = config
        self.task = None
        self.running = False

    def run(self):
        try:
            with nidaqmx.Task() as self.task:
                if self.config == 'RSE':
                    terminal_config = TerminalConfiguration.RSE
                elif self.config == 'NRSE':
                    terminal_config = TerminalConfiguration.NRSE
                elif self.config == 'DIFF':
                    terminal_config = TerminalConfiguration.DIFF
                elif self.config == 'PSEUDO_DIFF':
                    terminal_config = TerminalConfiguration.PSEUDO_DIFF
                else:
                    terminal_config = TerminalConfiguration.DEFAULT
                
                self.task.ai_channels.add_ai_voltage_chan(self.channel, terminal_config=terminal_config, min_val=-10, max_val=10)
                self.task.timing.cfg_samp_clk_timing(rate=self.sample_rate, sample_mode=AcquisitionType.CONTINUOUS)
                
                self.running = True
                self.start_time = time.time()
                self.task.start()
                
                while self.running:
                    data = self.task.read(number_of_samples_per_channel=self.buffer_size)
                    current_time = time.time()
                    if isinstance(data, (list, np.ndarray)):
                        data = np.array(data).flatten()
                        timestamps = np.linspace(current_time - self.start_time - len(data) / self.sample_rate,
                                                 current_time - self.start_time,
                                                 len(data))
                        self.update_plot.emit(timestamps, data)
                    
                    time.sleep(0.01)  # Reduced delay

        except Exception as e:
            print(f"Error in continuous AI sampling for {self.channel}: {str(e)}")

    def stop(self):
        print(f"Stopping AI task for {self.channel}")
        self.running = False
        
class WorkerSignals(QObject):
    finished = pyqtSignal()
    error = pyqtSignal(tuple)
    result = pyqtSignal(object)

class Worker(QRunnable):
    def __init__(self, fn, *args, **kwargs):
        super().__init__()
        self.fn = fn
        self.args = args
        self.kwargs = kwargs
        self.signals = WorkerSignals()

    def run(self):
        try:
            result = self.fn(*self.args, **self.kwargs)
        except:
            exctype, value = sys.exc_info()[:2]
            self.signals.error.emit((exctype, value, traceback.format_exc()))
        else:
            self.signals.result.emit(result)
        finally:
            self.signals.finished.emit()

class OutputThread(QThread):
    update_plot = pyqtSignal(str, np.ndarray, np.ndarray)
    error_occurred = pyqtSignal(str)
    finished = pyqtSignal()

    def __init__(self, channel, values, period, iterations, sample_rate):
        super().__init__()
        self.channel = channel
        self.values = values
        self.period = period
        self.iterations = iterations
        self.sample_rate = sample_rate

    def run(self):
        try:
            single_period_samples = len(self.values)
            total_samples = single_period_samples * self.iterations
            waveform = np.tile(self.values, self.iterations)
            
            total_duration = self.period * self.iterations
            timestamps = np.linspace(0, total_duration, total_samples)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(self.channel)
                task.timing.cfg_samp_clk_timing(rate=self.sample_rate, sample_mode=AcquisitionType.FINITE, samps_per_chan=total_samples)
                
                task.write(waveform, auto_start=True)
                
                self.update_plot.emit(self.channel, timestamps, waveform)
                
                task.wait_until_done(timeout=total_duration + 5.0)
            
            self.finished.emit()
        except Exception as e:
            self.error_occurred.emit(str(e))

class DAQmxController(QWidget):
    def __init__(self):
        super().__init__()
        self.threadpool = QThreadPool()

        self.ao_channels = ["Dev1/ao0", "Dev1/ao1", "Dev1/ao2", "Dev1/ao3"]

        self.ao_values = [np.array([]) for _ in range(len(self.ao_channels))]
        self.ao_timestamps = [np.array([]) for _ in range(len(self.ao_channels))]
        
        self.ao_period_textboxes = []
        self.ao_iterations_textboxes = []
        self.ao_file_paths = [None] * len(self.ao_channels)
        self.progress_bar = QProgressBar(self)
        self.progress_bar.setVisible(False)
        self.uploaded_data = {channel: None for channel in self.ao_channels}
        self.run_buttons = []
        self.ao_preview_plots = {}
        
        self.ai_channels = ["Dev1/ai0", "Dev1/ai1", "Dev1/ai2", "Dev1/ai3"]
        self.ai_tasks = {}
        self.ai_plots = {}
        self.ai_data = {channel: [[], []] for channel in self.ai_channels}  # (timestamps, values)
        self.ai_sampling_rates = [QLineEdit("10000") for _ in self.ai_channels]
        self.ai_curves = {}  # To store plot curves
        self.ai_config_dropdowns = {}
        self.ai_config_dropdowns = {}

        self.initUI()

    def initUI(self):
        main_layout = QVBoxLayout()
        # Create separate layouts for AO and AI controls
        ao_control_layout = QGridLayout()
        ai_control_layout = QGridLayout()
        
        # Create a horizontal layout to hold AO and AI control layouts side by side
        control_layout = QHBoxLayout()
        control_layout.addLayout(ao_control_layout)
        control_layout.addLayout(ai_control_layout)
        
        # Create a grid layout for all plots
        plot_layout = QGridLayout()
        plot_layout.setHorizontalSpacing(5)
        plot_layout.setVerticalSpacing(5)
        
        self.ao_labels = []
        self.ao_plots = {}
        self.ai_plots = {}

        # Set up AO controls and preview plots
        for i, ao_channel in enumerate(self.ao_channels):
            ao_label = QLabel(f"Analog Output {ao_channel}")
            ao_upload_button = QPushButton("Upload")
            ao_upload_button.clicked.connect(partial(self.upload_ao_values, ao_channel))

            ao_period_label = QLabel("Period (s):")
            ao_period_textbox = QLineEdit("0.005")
            ao_iterations_label = QLabel("Iterations:")
            ao_iterations_textbox = QLineEdit("1")

            ao_set_button = QPushButton("Set")
            ao_set_button.clicked.connect(partial(self.set_ao_settings, ao_channel))

            run_button = QPushButton("Run")
            run_button.clicked.connect(partial(self.run_ao_values, ao_channel))
            run_button.setEnabled(False)
            self.run_buttons.append(run_button)

            ao_reset_button = QPushButton("Reset")
            ao_reset_button.clicked.connect(partial(self.reset_ao_graph, ao_channel))

            ao_control_layout.addWidget(ao_label, i, 0)
            ao_control_layout.addWidget(ao_upload_button, i, 1)
            ao_control_layout.addWidget(ao_period_label, i, 2)
            ao_control_layout.addWidget(ao_period_textbox, i, 3)
            ao_control_layout.addWidget(ao_iterations_label, i, 4)
            ao_control_layout.addWidget(ao_iterations_textbox, i, 5)
            ao_control_layout.addWidget(ao_set_button, i, 6)
            ao_control_layout.addWidget(run_button, i, 7)
            ao_control_layout.addWidget(ao_reset_button, i, 8)

            self.ao_labels.append(ao_label)
            self.ao_period_textboxes.append(ao_period_textbox)
            self.ao_iterations_textboxes.append(ao_iterations_textbox)

            # Create and add AO preview plot
            preview_plot = pg.PlotWidget(title=f"Preview: {ao_channel}")
            preview_plot.setLabel('left', 'Voltage', units='V')
            preview_plot.setLabel('bottom', 'Sample')
            preview_plot.showGrid(x=True, y=True)
            preview_plot.setYRange(-10, 10, padding=0)
            preview_plot.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            preview_plot.setFixedSize(300, 200)
            self.ao_preview_plots[ao_channel] = preview_plot
            plot_layout.addWidget(preview_plot, i, 0)

            # Create and add AO plot
            plot_widget_ao = pg.PlotWidget(title=f"Analog Output Wave {ao_channel}")
            plot_widget_ao.setLabel('left', 'Voltage', units='V')
            plot_widget_ao.setLabel('bottom', 'Time', units='s')
            plot_widget_ao.showGrid(x=True, y=True)
            plot_widget_ao.setYRange(-10, 10, padding=0)
            plot_widget_ao.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            plot_widget_ao.setFixedSize(300, 200)
            self.ao_plots[ao_channel] = plot_widget_ao
            plot_layout.addWidget(plot_widget_ao, i, 1)

        # Set up AI controls and plots
        for i, ai_channel in enumerate(self.ai_channels):
            ai_label = QLabel(f"Analog Input {ai_channel}")
            ai_start_button = QPushButton("Start")
            ai_stop_button = QPushButton("Stop")
            ai_start_button.clicked.connect(partial(self.start_ai_task, ai_channel))
            ai_stop_button.clicked.connect(partial(self.stop_ai_task, ai_channel))

            ai_sampling_rate_label = QLabel("Sampling Rate:")
            ai_sampling_rate = self.ai_sampling_rates[i]

            ai_control_layout.addWidget(ai_label, i, 0)
            ai_control_layout.addWidget(ai_sampling_rate_label, i, 1)
            ai_control_layout.addWidget(ai_sampling_rate, i, 2)
            ai_control_layout.addWidget(ai_start_button, i, 3)
            ai_control_layout.addWidget(ai_stop_button, i, 4)

            # Create and add AI plot
            plot_widget_ai = pg.PlotWidget(title=f"Analog Input {ai_channel}")
            plot_widget_ai.setLabel('left', 'Voltage', units='V')
            plot_widget_ai.setLabel('bottom', 'Time', units='s')
            plot_widget_ai.showGrid(x=True, y=True)
            plot_widget_ai.setYRange(5.5, 6.5)  # Set initial y-range around 6V
            plot_widget_ai.setXRange(0, 1)  # Set initial x-range to 1 second
            plot_widget_ai.setFixedSize(300, 200)  # Add this line
            self.ai_plots[ai_channel] = plot_widget_ai
            self.ai_curves[ai_channel] = plot_widget_ai.plot(pen='y')
            plot_layout.addWidget(plot_widget_ai, i, 2)
            
            # In the initUI method, where you set up AI controls
            ai_config_label = QLabel("Input Config:")
            ai_config_dropdown = QComboBox()
            ai_config_dropdown.addItems(["RSE", "NRSE", "Differential", "PSEUDO_DIFF"])
            ai_config_dropdown.setCurrentText("RSE")  # Set default to RSE
            ai_config_dropdown.currentTextChanged.connect(partial(self.change_ai_config, ai_channel))

            ai_control_layout.addWidget(ai_config_label, i, 5)
            ai_control_layout.addWidget(ai_config_dropdown, i, 6)
            
            ai_set_button = QPushButton("Set")
            ai_set_button.clicked.connect(partial(self.set_ai_config, ai_channel))
            ai_control_layout.addWidget(ai_set_button, i, 7)
            
            ai_reset_button = QPushButton("Reset")
            ai_reset_button.clicked.connect(partial(self.reset_ai_graph, ai_channel))
            ai_control_layout.addWidget(ai_reset_button, i, 8)

            # Store the dropdown in a dictionary for later access
            self.ai_config_dropdowns[ai_channel] = ai_config_dropdown

        # Create scroll areas for controls and plots
        control_widget = QWidget()
        control_widget.setLayout(control_layout)
        control_scroll_area = QScrollArea()
        control_scroll_area.setWidget(control_widget)
        control_scroll_area.setWidgetResizable(True)
        control_scroll_area.setFixedHeight(200)

        plot_widget = QWidget()
        plot_widget.setLayout(plot_layout)
        plot_scroll_area = QScrollArea()
        plot_scroll_area.setWidget(plot_widget)
        plot_scroll_area.setWidgetResizable(True)

        main_layout.addWidget(control_scroll_area)
        main_layout.addWidget(plot_scroll_area)
        
        self.setLayout(main_layout)
        self.setWindowTitle('DAQmx Analog IO Controller')
        self.showMaximized()
     
    def calculate_max_sampling_rate(self):
        active_channels = len(self.ai_tasks)
        if active_channels == 1:
            return 600000  # Max for single channel
        else:
            return min(500000 // (active_channels + 1), 600000)  # Respect multi-channel limit
       
    def change_ai_config(self, channel, config):
        if channel in self.ai_tasks:
            self.stop_ai_task(channel)
        
        # Restart the task with the new configuration
        self.start_ai_task(channel, config)

    def reset_ai_graph(self, channel):
        self.ai_data[channel] = [[], []]
        self.ai_curves[channel].setData([], [])
        self.ai_plots[channel].setYRange(-10, 10)
        self.ai_plots[channel].setXRange(0, 6)
    
    def set_ai_config(self, channel):
        config = self.ai_config_dropdowns[channel].currentText()
        if channel in self.ai_tasks:
            self.stop_ai_task(channel)
        self.start_ai_task(channel, config)
    
    def update_all_ai_tasks(self):
        max_rate = self.calculate_max_sampling_rate()
        for channel, task in self.ai_tasks.items():
            if task.isRunning():
                new_rate = min(float(self.ai_sampling_rates[self.ai_channels.index(channel)].text()), max_rate)
                task.sample_rate = new_rate
                task.buffer_size = int(new_rate * 0.1)
                print(f"Updated {channel} to sample rate {new_rate}")
                
    def start_ai_task(self, channel, config=None):
        print(f"Attempting to start AI task for {channel}")
        
        max_rate = self.calculate_max_sampling_rate()
        index = self.ai_channels.index(channel)
        requested_rate = float(self.ai_sampling_rates[index].text())
        sample_rate = min(requested_rate, max_rate)
        
        buffer_size = int(sample_rate * 0.1)  # 0.1 seconds of data

        if config is None:
            config = self.ai_config_dropdowns[channel].currentText()
        
        # Ensure config is a string
        config = str(config)

        task = AnalogInputTask(channel, sample_rate, buffer_size, config)
        task.update_plot.connect(partial(self.update_ai_plot, channel))
        task.start()

        self.ai_tasks[channel] = task
        print(f"Started AI task for {channel} with sample rate {sample_rate} and configuration {config}")
        
        # Update all other running tasks
        self.update_all_ai_tasks()

    def stop_ai_task(self, channel):
        print(f"Attempting to stop AI task for {channel}")
        if channel in self.ai_tasks:
            self.ai_tasks[channel].stop()
            self.ai_tasks[channel].wait()
            del self.ai_tasks[channel]
            print(f"AI task stopped for {channel}")
            QMessageBox.information(self, "Info", f"Stopped continuous sampling for {channel}")
            
            # Update other tasks
            self.update_all_ai_tasks()
        else:
            print(f"No AI task running for {channel}")
            QMessageBox.warning(self, "Warning", f"No continuous sampling running for {channel}")

    def update_ai_plot(self, channel, timestamps, values):
        try:
            if isinstance(timestamps, np.ndarray) and isinstance(values, np.ndarray):
                if len(timestamps) > 0 and len(values) > 0:
                    self.ai_data[channel][0].extend(timestamps)
                    self.ai_data[channel][1].extend(values)
                    
                    # Keep only the last 60000 points (6 seconds at 10kHz)
                    if len(self.ai_data[channel][0]) > 60000:
                        self.ai_data[channel][0] = self.ai_data[channel][0][-60000:]
                        self.ai_data[channel][1] = self.ai_data[channel][1][-60000:]
                    
                    self.ai_curves[channel].setData(self.ai_data[channel][0], self.ai_data[channel][1])
                    
                    # Update axis ranges
                    x_min = max(0, self.ai_data[channel][0][-1] - 6)
                    x_max = self.ai_data[channel][0][-1]
                    self.ai_plots[channel].setXRange(x_min, x_max)
                    y_min, y_max = min(self.ai_data[channel][1]), max(self.ai_data[channel][1])
                    y_range = y_max - y_min
                    self.ai_plots[channel].setYRange(y_min - 0.1 * y_range, y_max + 0.1 * y_range)
                    
                    print(f"Updating plot for {channel}: min={y_min:.3f}, max={y_max:.3f}, len={len(self.ai_data[channel][0])}")
                else:
                    print(f"Received empty data for {channel}")
            else:
                print(f"Unexpected data format for {channel}: timestamps={type(timestamps)}, values={type(values)}")
        except Exception as e:
            print(f"Error updating AI plot: {str(e)}")
        
    def set_ao_value(self, channel, textbox):
        try:
            value = float(textbox.text())
            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.write(value)
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error setting AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def sweep_ao_voltage(self, channel, start_voltage, end_voltage, voltage_step, iterations):
        try:
            start = float(start_voltage.text())
            end = float(end_voltage.text())
            step = float(voltage_step.text())
            num_iterations = int(iterations.text())

            if start >= end:
                raise ValueError("Start voltage must be less than end voltage.")
            if step <= 0:
                raise ValueError("Voltage step must be greater than zero.")
            if num_iterations <= 0:
                raise ValueError("Number of iterations must be greater than zero.")

            voltages = np.arange(start, end + step, step)
            waveform = np.tile(voltages, num_iterations)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.timing.cfg_samp_clk_timing(rate=1000, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                task.write(waveform, auto_start=True)

                index = self.ao_channels.index(channel)
                timestamps = np.linspace(0, len(waveform) / 1000, len(waveform))
                for i in range(len(waveform)):
                    self.ao_values[index].append(waveform[i])
                    self.ao_timestamps[index].append(timestamps[i])
                    self.ao_plots[channel].clear()
                    self.ao_plots[channel].plot(self.ao_timestamps[index], self.ao_values[index])
                    QApplication.processEvents()

                task.wait_until_done()
                task.stop()

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error sweeping AO voltage: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def set_ao_settings(self, channel):
        try:
            index = self.ao_channels.index(channel)
            period = float(self.ao_period_textboxes[index].text())
            iterations = int(self.ao_iterations_textboxes[index].text())

            if period <= 0:
                raise ValueError("Period must be greater than zero.")
            if iterations <= 0:
                raise ValueError("Iterations must be greater than zero.")

            if self.uploaded_data[channel] is None:
                raise ValueError("No data uploaded for this channel.")

            values = self.uploaded_data[channel]
            samples = len(values)
            t = np.linspace(0, period * iterations, samples * iterations)
            preview_waveform = np.tile(values, iterations)

            self.ao_preview_plots[channel].clear()
            self.ao_preview_plots[channel].plot(t, preview_waveform)
            self.ao_preview_plots[channel].setLabel('bottom', 'Time', units='s')

            QMessageBox.information(self, "Settings Applied", f"Settings applied for {channel}. Check the preview graph.")

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Error setting AO values: {str(e)}")
            
    def upload_ao_values(self, channel):
        try:
            file_dialog = QFileDialog()
            file_path, _ = file_dialog.getOpenFileName(self, "Open Text File", "", "Text Files (*.txt)")
            if file_path:
                worker = Worker(self._load_file, file_path, channel)
                worker.signals.result.connect(self.update_preview)
                worker.signals.error.connect(self.handle_error)
                self.threadpool.start(worker)
        except Exception as e:
            self.handle_error(("Error", e, ""))

    def _load_file(self, file_path, channel):
        with open(file_path, "r") as file:
            values = [float(line.strip()) for line in file.readlines()]
        return channel, values

    def update_preview(self, result):
        channel, values = result
        self.uploaded_data[channel] = values
        index = self.ao_channels.index(channel)
        
        self.ao_preview_plots[channel].clear()
        self.ao_preview_plots[channel].plot(range(len(values)), values)
        
        self.run_buttons[index].setEnabled(True)
    
    def closeEvent(self, event):
        for channel in list(self.ai_tasks.keys()):
            self.stop_ai_task(channel)
        for task in nidaqmx.system.System().tasks:
            try:
                task.close()
            except:
                pass
        event.accept()

    def run_ao_values(self, channel):
        if self.uploaded_data[channel] is None:
            QMessageBox.warning(self, "Warning", "No data uploaded for this channel.")
            return
        
        index = self.ao_channels.index(channel)
        values = self.uploaded_data[channel]
        period = float(self.ao_period_textboxes[index].text())
        iterations = int(self.ao_iterations_textboxes[index].text())

        sample_rate = len(values) / period
        samples_per_channel = len(values) * iterations

        waveform = np.tile(values, iterations)

        try:
            with nidaqmx.Task() as ao_task:
                ao_task.ao_channels.add_ao_voltage_chan(channel)
                ao_task.timing.cfg_samp_clk_timing(rate=sample_rate,
                                                sample_mode=AcquisitionType.FINITE,
                                                samps_per_chan=samples_per_channel)

                ao_writer = AnalogMultiChannelWriter(ao_task.out_stream)

                ao_writer.write_many_sample(waveform.reshape(1, -1))

                start_time = time.perf_counter()
                ao_task.start()

                ao_task.wait_until_done(timeout=period * iterations + 5.0)

                end_time = time.perf_counter()

            timestamps = np.linspace(0, period * iterations, samples_per_channel)

            self.update_plot(channel, timestamps, waveform)

            expected_duration = period * iterations
            actual_duration = end_time - start_time
            timing_error = abs(actual_duration - expected_duration)

            info_message = (f"Waveform output completed for channel {channel}\n"
                            f"Expected duration: {expected_duration:.6f}s\n"
                            f"Actual duration: {actual_duration:.6f}s\n"
                            f"Timing error: {timing_error:.6f}s")
            
            QMessageBox.information(self, "Success", info_message)

        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error during AO operation: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")
        
    def update_plots(self, channel, ao_timestamps, ao_values):
        try:
            # Update AO plot
            self.ao_plots[channel].clear()
            self.ao_plots[channel].plot(ao_timestamps, ao_values)
            
            QApplication.processEvents()
        except Exception as e:
            print(f"Error updating plots: {str(e)}")
    
    def update_progress(self, value):
        self.progress_bar.setValue(value)

    def update_plot(self, channel, timestamps, values):
        self.ao_plots[channel].clear()
        self.ao_plots[channel].plot(timestamps * 1000, values)  # Convert timestamps to milliseconds for display
        self.ao_plots[channel].setLabel('bottom', 'Time', units='ms')
        self.ao_plots[channel].setXRange(0, timestamps[-1] * 1000)  # Ensure full range is visible
        QApplication.processEvents()
        
    def handle_thread_error(self, error_message):
        QMessageBox.critical(self, "Error", f"An error occurred: {error_message}")

    def output_finished(self, channel):
        QMessageBox.information(self, "Success", f"Waveform output completed for channel {channel}")
        
    def handle_error(self, error_info):
        QMessageBox.critical(self, "Error", str(error_info[1]))
                            
    def reset_ao_graph(self, channel):
        index = self.ao_channels.index(channel)
        self.ao_values[index] = np.array([])
        self.ao_timestamps[index] = np.array([])
        self.ao_plots[channel].clear()
    
    def read_ao_value(self, channel, index):
        try:
            with nidaqmx.Task() as task:
                task.ai_channels.add_ai_voltage_chan(channel,min_val=-10,max_val=10)
                value = task.read()
                self.ao_read_labels[index].setText(f"{value[0]:.2f}")
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error reading AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = DAQmxController()
    sys.exit(app.exec_())

# The multichannel not working.


In [ ]:
# analog output and input tasks untied from the main thread. The 2 ports act independently.
# Possible for more than 1 analog input port to be activated at a single time during sampling.

import sys
import numpy as np
import time
from functools import partial
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QHBoxLayout, QGridLayout, 
    QScrollArea, QComboBox, QLineEdit, QFileDialog, QMessageBox, QProgressDialog, QProgressBar
)
from PyQt5.QtCore import Qt, QThread, pyqtSignal, QRunnable, QThreadPool, QObject
import pyqtgraph as pg
import nidaqmx
from nidaqmx.constants import AcquisitionType, Edge, TerminalConfiguration
from nidaqmx.stream_writers import AnalogMultiChannelWriter
from PyQt5.QtCore import QTimer
import threading
import time
class AnalogInputTask(QThread):
    update_plot = pyqtSignal(dict)

    def __init__(self, channels, sample_rate, buffer_size, configs):
        super().__init__()
        self.channels = channels
        self.sample_rate = sample_rate
        self.buffer_size = buffer_size
        self.configs = configs
        self.task = None
        self.running = False

    def run(self):
        try:
            with nidaqmx.Task() as self.task:
                for channel, config in zip(self.channels, self.configs):
                    if config == 'RSE':
                        terminal_config = TerminalConfiguration.RSE
                    elif config == 'NRSE':
                        terminal_config = TerminalConfiguration.NRSE
                    elif config == 'DIFF':
                        terminal_config = TerminalConfiguration.DIFF
                    elif config == 'PSEUDO_DIFF':
                        terminal_config = TerminalConfiguration.PSEUDO_DIFF
                    else:
                        terminal_config = TerminalConfiguration.DEFAULT
                    
                    self.task.ai_channels.add_ai_voltage_chan(channel, terminal_config=terminal_config, min_val=-10, max_val=10)
                
                self.task.timing.cfg_samp_clk_timing(rate=self.sample_rate, sample_mode=AcquisitionType.CONTINUOUS)
                
                self.running = True
                self.start_time = time.time()
                self.task.start()
                
                while self.running:
                    data = self.task.read(number_of_samples_per_channel=self.buffer_size)
                    current_time = time.time()
                    data = np.array(data)
                    if data.ndim == 1:
                        data = data.reshape(1, -1)
                    timestamps = np.linspace(current_time - self.start_time - data.shape[1] / self.sample_rate,
                                             current_time - self.start_time,
                                             data.shape[1])
                    data_dict = {channel: data[i, :] for i, channel in enumerate(self.channels)}
                    data_dict['timestamps'] = timestamps
                    self.update_plot.emit(data_dict)
                    
                    time.sleep(0.01)  # Reduced delay

        except Exception as e:
            print(f"Error in continuous AI sampling: {str(e)}")

    def stop(self):
        print(f"Stopping AI task for channels: {', '.join(self.channels)}")
        self.running = False
        
class WorkerSignals(QObject):
    finished = pyqtSignal()
    error = pyqtSignal(tuple)
    result = pyqtSignal(object)

class Worker(QRunnable):
    def __init__(self, fn, *args, **kwargs):
        super().__init__()
        self.fn = fn
        self.args = args
        self.kwargs = kwargs
        self.signals = WorkerSignals()

    def run(self):
        try:
            result = self.fn(*self.args, **self.kwargs)
        except:
            exctype, value = sys.exc_info()[:2]
            self.signals.error.emit((exctype, value, traceback.format_exc()))
        else:
            self.signals.result.emit(result)
        finally:
            self.signals.finished.emit()

class OutputThread(QThread):
    update_plot = pyqtSignal(str, np.ndarray, np.ndarray)
    error_occurred = pyqtSignal(str)
    finished = pyqtSignal()

    def __init__(self, channel, values, period, iterations, sample_rate):
        super().__init__()
        self.channel = channel
        self.values = values
        self.period = period
        self.iterations = iterations
        self.sample_rate = sample_rate

    def run(self):
        try:
            single_period_samples = len(self.values)
            total_samples = single_period_samples * self.iterations
            waveform = np.tile(self.values, self.iterations)
            
            total_duration = self.period * self.iterations
            timestamps = np.linspace(0, total_duration, total_samples)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(self.channel)
                task.timing.cfg_samp_clk_timing(rate=self.sample_rate, sample_mode=AcquisitionType.FINITE, samps_per_chan=total_samples)
                
                task.write(waveform, auto_start=True)
                
                self.update_plot.emit(self.channel, timestamps, waveform)
                
                task.wait_until_done(timeout=total_duration + 5.0)
            
            self.finished.emit()
        except Exception as e:
            self.error_occurred.emit(str(e))

class DAQmxController(QWidget):
    def __init__(self):
        super().__init__()
        self.threadpool = QThreadPool()
        

        self.ao_channels = ["Dev1/ao0", "Dev1/ao1", "Dev1/ao2", "Dev1/ao3"]

        self.ao_values = [np.array([]) for _ in range(len(self.ao_channels))]
        self.ao_timestamps = [np.array([]) for _ in range(len(self.ao_channels))]
        
        self.ao_period_textboxes = []
        self.ao_iterations_textboxes = []
        self.ao_file_paths = [None] * len(self.ao_channels)
        self.progress_bar = QProgressBar(self)
        self.progress_bar.setVisible(False)
        self.uploaded_data = {channel: None for channel in self.ao_channels}
        self.run_buttons = []
        self.ao_preview_plots = {}
        
        self.ai_tasks = {}
        self.active_ai_channels = set()
        self.ai_channels = ["Dev1/ai0", "Dev1/ai1", "Dev1/ai2", "Dev1/ai3"]
        self.ai_tasks = {}
        self.ai_plots = {}
        self.ai_data = {channel: [[], []] for channel in self.ai_channels}  # (timestamps, values)
        self.ai_sampling_rates = [QLineEdit("10000") for _ in self.ai_channels]
        self.ai_curves = {}  # To store plot curves
        self.ai_config_dropdowns = {}
        self.ai_config_dropdowns = {}

        self.initUI()

    def initUI(self):
        main_layout = QVBoxLayout()
        # Create separate layouts for AO and AI controls
        ao_control_layout = QGridLayout()
        ai_control_layout = QGridLayout()
        
        # Create a horizontal layout to hold AO and AI control layouts side by side
        control_layout = QHBoxLayout()
        control_layout.addLayout(ao_control_layout)
        control_layout.addLayout(ai_control_layout)
        
        # Create a grid layout for all plots
        plot_layout = QGridLayout()
        plot_layout.setHorizontalSpacing(5)
        plot_layout.setVerticalSpacing(5)
        
        self.ao_labels = []
        self.ao_plots = {}
        self.ai_plots = {}

        # Set up AO controls and preview plots
        for i, ao_channel in enumerate(self.ao_channels):
            ao_label = QLabel(f"Analog Output {ao_channel}")
            ao_upload_button = QPushButton("Upload")
            ao_upload_button.clicked.connect(partial(self.upload_ao_values, ao_channel))

            ao_period_label = QLabel("Period (s):")
            ao_period_textbox = QLineEdit("0.005")
            ao_iterations_label = QLabel("Iterations:")
            ao_iterations_textbox = QLineEdit("1")

            ao_set_button = QPushButton("Set")
            ao_set_button.clicked.connect(partial(self.set_ao_settings, ao_channel))

            run_button = QPushButton("Run")
            run_button.clicked.connect(partial(self.run_ao_values, ao_channel))
            run_button.setEnabled(False)
            self.run_buttons.append(run_button)

            ao_reset_button = QPushButton("Reset")
            ao_reset_button.clicked.connect(partial(self.reset_ao_graph, ao_channel))

            ao_control_layout.addWidget(ao_label, i, 0)
            ao_control_layout.addWidget(ao_upload_button, i, 1)
            ao_control_layout.addWidget(ao_period_label, i, 2)
            ao_control_layout.addWidget(ao_period_textbox, i, 3)
            ao_control_layout.addWidget(ao_iterations_label, i, 4)
            ao_control_layout.addWidget(ao_iterations_textbox, i, 5)
            ao_control_layout.addWidget(ao_set_button, i, 6)
            ao_control_layout.addWidget(run_button, i, 7)
            ao_control_layout.addWidget(ao_reset_button, i, 8)

            self.ao_labels.append(ao_label)
            self.ao_period_textboxes.append(ao_period_textbox)
            self.ao_iterations_textboxes.append(ao_iterations_textbox)

            # Create and add AO preview plot
            preview_plot = pg.PlotWidget(title=f"Preview: {ao_channel}")
            preview_plot.setLabel('left', 'Voltage', units='V')
            preview_plot.setLabel('bottom', 'Sample')
            preview_plot.showGrid(x=True, y=True)
            preview_plot.setYRange(-10, 10, padding=0)
            preview_plot.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            preview_plot.setFixedSize(300, 200)
            self.ao_preview_plots[ao_channel] = preview_plot
            plot_layout.addWidget(preview_plot, i, 0)

            # Create and add AO plot
            plot_widget_ao = pg.PlotWidget(title=f"Analog Output Wave {ao_channel}")
            plot_widget_ao.setLabel('left', 'Voltage', units='V')
            plot_widget_ao.setLabel('bottom', 'Time', units='s')
            plot_widget_ao.showGrid(x=True, y=True)
            plot_widget_ao.setYRange(-10, 10, padding=0)
            plot_widget_ao.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            plot_widget_ao.setFixedSize(300, 200)
            self.ao_plots[ao_channel] = plot_widget_ao
            plot_layout.addWidget(plot_widget_ao, i, 1)

        # Set up AI controls and plots
        for i, ai_channel in enumerate(self.ai_channels):
            ai_label = QLabel(f"Analog Input {ai_channel}")
            ai_start_button = QPushButton("Start")
            ai_stop_button = QPushButton("Stop")
            ai_start_button.clicked.connect(partial(self.start_ai_task, ai_channel))
            ai_stop_button.clicked.connect(partial(self.stop_ai_task, ai_channel))

            ai_sampling_rate_label = QLabel("Sampling Rate:")
            ai_sampling_rate = QLineEdit("10000")  # Default to 10 kHz
            self.ai_sampling_rates.append(ai_sampling_rate)

            ai_control_layout.addWidget(ai_label, i, 0)
            ai_control_layout.addWidget(ai_sampling_rate_label, i, 1)
            ai_control_layout.addWidget(ai_sampling_rate, i, 2)
            ai_control_layout.addWidget(ai_start_button, i, 3)
            ai_control_layout.addWidget(ai_stop_button, i, 4)

            # Create and add AI plot
            plot_widget_ai = pg.PlotWidget(title=f"Analog Input {ai_channel}")
            plot_widget_ai.setLabel('left', 'Voltage', units='V')
            plot_widget_ai.setLabel('bottom', 'Time', units='s')
            plot_widget_ai.showGrid(x=True, y=True)
            plot_widget_ai.setYRange(5.5, 6.5)  # Set initial y-range around 6V
            plot_widget_ai.setXRange(0, 1)  # Set initial x-range to 1 second
            plot_widget_ai.setFixedSize(300, 200)  # Add this line
            self.ai_plots[ai_channel] = plot_widget_ai
            self.ai_curves[ai_channel] = plot_widget_ai.plot(pen='y')
            plot_layout.addWidget(plot_widget_ai, i, 2)
            
            # In the initUI method, where you set up AI controls
            ai_config_label = QLabel("Input Config:")
            ai_config_dropdown = QComboBox()
            ai_config_dropdown.addItems(["RSE", "NRSE", "Differential", "PSEUDO_DIFF"])
            ai_config_dropdown.setCurrentText("RSE")  # Set default to RSE
            ai_config_dropdown.currentTextChanged.connect(partial(self.change_ai_config, ai_channel))

            ai_control_layout.addWidget(ai_config_label, i, 5)
            ai_control_layout.addWidget(ai_config_dropdown, i, 6)
            
            ai_set_button = QPushButton("Set")
            ai_set_button.clicked.connect(partial(self.set_ai_config, ai_channel))
            ai_control_layout.addWidget(ai_set_button, i, 7)
            
            ai_reset_button = QPushButton("Reset")
            ai_reset_button.clicked.connect(partial(self.reset_ai_graph, ai_channel))
            ai_control_layout.addWidget(ai_reset_button, i, 8)

            # Store the dropdown in a dictionary for later access
            self.ai_config_dropdowns[ai_channel] = ai_config_dropdown

        # Create scroll areas for controls and plots
        control_widget = QWidget()
        control_widget.setLayout(control_layout)
        control_scroll_area = QScrollArea()
        control_scroll_area.setWidget(control_widget)
        control_scroll_area.setWidgetResizable(True)
        control_scroll_area.setFixedHeight(200)

        plot_widget = QWidget()
        plot_widget.setLayout(plot_layout)
        plot_scroll_area = QScrollArea()
        plot_scroll_area.setWidget(plot_widget)
        plot_scroll_area.setWidgetResizable(True)

        main_layout.addWidget(control_scroll_area)
        main_layout.addWidget(plot_scroll_area)
        
        self.setLayout(main_layout)
        self.setWindowTitle('DAQmx Analog IO Controller')
        self.showMaximized()
       
    def change_ai_config(self, channel, config):
        if channel in self.ai_tasks:
            self.stop_ai_task(channel)
        self.start_ai_task(channel)

    def reset_ai_graph(self, channel):
        self.ai_data[channel] = [[], []]
        self.ai_curves[channel].setData([], [])
        self.ai_plots[channel].setYRange(-10, 10)
        self.ai_plots[channel].setXRange(0, 6)
    
    def set_ai_config(self, channel):
        if channel in self.ai_tasks:
            self.stop_ai_task(channel)
        self.start_ai_task(channel)
    
    def update_all_ai_tasks(self):
        if not self.ai_tasks:
            print("No active AI tasks to update.")
            return

        max_rate = self.calculate_max_sampling_rate()
        for channel, task in self.ai_tasks.items():
            new_rate = min(float(self.ai_sampling_rates[self.ai_channels.index(channel)].text()), max_rate)
            task.update_sample_rate(new_rate)
            print(f"Updated {channel} to sample rate {new_rate}")
                
    def start_ai_task(self, channel):
        print(f"Attempting to start AI task for {channel}")
        
        if channel in self.active_ai_channels:
            print(f"Channel {channel} is already running.")
            return

        self.active_ai_channels.add(channel)
        
        # Update or create the task for this channel
        max_rate = self.calculate_max_sampling_rate()
        sample_rate = min(max_rate, float(self.ai_sampling_rates[self.ai_channels.index(channel)].text()))
        buffer_size = int(sample_rate * 0.1)  # 0.1 seconds of data
        config = self.ai_config_dropdowns[channel].currentText()

        if not self.ai_tasks:
            # If no task exists, create a new one
            task = AnalogInputTask([channel], sample_rate, buffer_size, [config])
            task.update_plot.connect(self.update_ai_plots)
            task.start()
            self.ai_tasks[channel] = task
        else:
            # If a task exists, add this channel to it
            existing_task = next(iter(self.ai_tasks.values()))
            existing_task.channels.append(channel)
            existing_task.configs.append(config)
            existing_task.stop()
            existing_task.wait()
            
            # Recreate the task with the updated channels
            new_task = AnalogInputTask(existing_task.channels, sample_rate, buffer_size, existing_task.configs)
            new_task.update_plot.connect(self.update_ai_plots)
            new_task.start()
            
            # Update all channel references to the new task
            for ch in existing_task.channels:
                self.ai_tasks[ch] = new_task

        print(f"Started AI task for channel: {channel} with sample rate {sample_rate}")

    def stop_ai_task(self, channel):
        if channel not in self.active_ai_channels:
            print(f"Channel {channel} is not running.")
            return

        self.active_ai_channels.remove(channel)
        
        if channel in self.ai_tasks:
            task = self.ai_tasks[channel]
            task.channels.remove(channel)
            task.configs.pop(task.channels.index(channel))
            
            if not task.channels:
                # If no channels left, stop and remove the task
                task.stop()
                task.wait()
                self.ai_tasks.clear()
            else:
                # Recreate the task with the remaining channels
                sample_rate = task.sample_rate
                buffer_size = task.buffer_size
                new_task = AnalogInputTask(task.channels, sample_rate, buffer_size, task.configs)
                new_task.update_plot.connect(self.update_ai_plots)
                new_task.start()
                
                # Update all channel references to the new task
                for ch in task.channels:
                    self.ai_tasks[ch] = new_task
                
                # Stop the old task
                task.stop()
                task.wait()

            del self.ai_tasks[channel]
        
        print(f"Stopped AI task for channel: {channel}")

    def update_ai_plots(self, data_dict):
        timestamps = data_dict['timestamps']
        for channel in self.active_ai_channels:
            if channel in data_dict:
                values = data_dict[channel]
                
                # Ensure timestamps and values are arrays
                if not isinstance(timestamps, np.ndarray):
                    timestamps = np.array([timestamps])
                if not isinstance(values, np.ndarray):
                    values = np.array([values])
                
                self.ai_data[channel][0].extend(timestamps)
                self.ai_data[channel][1].extend(values)
                
                # Keep only the last 60000 points (6 seconds at 10kHz)
                if len(self.ai_data[channel][0]) > 60000:
                    self.ai_data[channel][0] = self.ai_data[channel][0][-60000:]
                    self.ai_data[channel][1] = self.ai_data[channel][1][-60000:]
                
                self.ai_curves[channel].setData(self.ai_data[channel][0], self.ai_data[channel][1])
                
                # Update axis ranges
                if len(self.ai_data[channel][0]) > 0:  # Check if we have any data
                    x_min = max(0, self.ai_data[channel][0][-1] - 6)
                    x_max = self.ai_data[channel][0][-1]
                    self.ai_plots[channel].setXRange(x_min, x_max)
                    y_min, y_max = min(self.ai_data[channel][1]), max(self.ai_data[channel][1])
                    y_range = y_max - y_min
                    self.ai_plots[channel].setYRange(y_min - 0.1 * y_range, y_max + 0.1 * y_range)
                
                print(f"Updating plot for {channel}: min={np.min(values):.3f}, max={np.max(values):.3f}, len={len(self.ai_data[channel][0])}")

    def calculate_max_sampling_rate(self):
        active_channels = len(self.active_ai_channels)
        if active_channels == 0:
            return 600000  # Return max rate if no channels are active
        elif active_channels == 1:
            return 600000  # Max for single channel
        else:
            return min(500000 // active_channels, 600000)  # Respect multi-channel limit
        
    def set_ao_value(self, channel, textbox):
        try:
            value = float(textbox.text())
            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.write(value)
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error setting AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def sweep_ao_voltage(self, channel, start_voltage, end_voltage, voltage_step, iterations):
        try:
            start = float(start_voltage.text())
            end = float(end_voltage.text())
            step = float(voltage_step.text())
            num_iterations = int(iterations.text())

            if start >= end:
                raise ValueError("Start voltage must be less than end voltage.")
            if step <= 0:
                raise ValueError("Voltage step must be greater than zero.")
            if num_iterations <= 0:
                raise ValueError("Number of iterations must be greater than zero.")

            voltages = np.arange(start, end + step, step)
            waveform = np.tile(voltages, num_iterations)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.timing.cfg_samp_clk_timing(rate=1000, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                task.write(waveform, auto_start=True)

                index = self.ao_channels.index(channel)
                timestamps = np.linspace(0, len(waveform) / 1000, len(waveform))
                for i in range(len(waveform)):
                    self.ao_values[index].append(waveform[i])
                    self.ao_timestamps[index].append(timestamps[i])
                    self.ao_plots[channel].clear()
                    self.ao_plots[channel].plot(self.ao_timestamps[index], self.ao_values[index])
                    QApplication.processEvents()

                task.wait_until_done()
                task.stop()

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error sweeping AO voltage: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def set_ao_settings(self, channel):
        try:
            index = self.ao_channels.index(channel)
            period = float(self.ao_period_textboxes[index].text())
            iterations = int(self.ao_iterations_textboxes[index].text())

            if period <= 0:
                raise ValueError("Period must be greater than zero.")
            if iterations <= 0:
                raise ValueError("Iterations must be greater than zero.")

            if self.uploaded_data[channel] is None:
                raise ValueError("No data uploaded for this channel.")

            values = self.uploaded_data[channel]
            samples = len(values)
            t = np.linspace(0, period * iterations, samples * iterations)
            preview_waveform = np.tile(values, iterations)

            self.ao_preview_plots[channel].clear()
            self.ao_preview_plots[channel].plot(t, preview_waveform)
            self.ao_preview_plots[channel].setLabel('bottom', 'Time', units='s')

            QMessageBox.information(self, "Settings Applied", f"Settings applied for {channel}. Check the preview graph.")

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Error setting AO values: {str(e)}")
            
    def upload_ao_values(self, channel):
        try:
            file_dialog = QFileDialog()
            file_path, _ = file_dialog.getOpenFileName(self, "Open Text File", "", "Text Files (*.txt)")
            if file_path:
                worker = Worker(self._load_file, file_path, channel)
                worker.signals.result.connect(self.update_preview)
                worker.signals.error.connect(self.handle_error)
                self.threadpool.start(worker)
        except Exception as e:
            self.handle_error(("Error", e, ""))

    def _load_file(self, file_path, channel):
        with open(file_path, "r") as file:
            values = [float(line.strip()) for line in file.readlines()]
        return channel, values

    def update_preview(self, result):
        channel, values = result
        self.uploaded_data[channel] = values
        index = self.ao_channels.index(channel)
        
        self.ao_preview_plots[channel].clear()
        self.ao_preview_plots[channel].plot(range(len(values)), values)
        
        self.run_buttons[index].setEnabled(True)
    
    def closeEvent(self, event):
        for channel in list(self.ai_tasks.keys()):
            self.stop_ai_task(channel)
        for task in nidaqmx.system.System().tasks:
            try:
                task.close()
            except:
                pass
        event.accept()

    def run_ao_values(self, channel):
        if self.uploaded_data[channel] is None:
            QMessageBox.warning(self, "Warning", "No data uploaded for this channel.")
            return
        
        index = self.ao_channels.index(channel)
        values = self.uploaded_data[channel]
        period = float(self.ao_period_textboxes[index].text())
        iterations = int(self.ao_iterations_textboxes[index].text())

        sample_rate = len(values) / period
        samples_per_channel = len(values) * iterations

        waveform = np.tile(values, iterations)

        try:
            with nidaqmx.Task() as ao_task:
                ao_task.ao_channels.add_ao_voltage_chan(channel)
                ao_task.timing.cfg_samp_clk_timing(rate=sample_rate,
                                                sample_mode=AcquisitionType.FINITE,
                                                samps_per_chan=samples_per_channel)

                ao_writer = AnalogMultiChannelWriter(ao_task.out_stream)

                ao_writer.write_many_sample(waveform.reshape(1, -1))

                start_time = time.perf_counter()
                ao_task.start()

                ao_task.wait_until_done(timeout=period * iterations + 5.0)

                end_time = time.perf_counter()

            timestamps = np.linspace(0, period * iterations, samples_per_channel)

            self.update_plot(channel, timestamps, waveform)

            expected_duration = period * iterations
            actual_duration = end_time - start_time
            timing_error = abs(actual_duration - expected_duration)

            info_message = (f"Waveform output completed for channel {channel}\n"
                            f"Expected duration: {expected_duration:.6f}s\n"
                            f"Actual duration: {actual_duration:.6f}s\n"
                            f"Timing error: {timing_error:.6f}s")
            
            QMessageBox.information(self, "Success", info_message)

        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error during AO operation: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")
        
    def update_plots(self, channel, ao_timestamps, ao_values):
        try:
            # Update AO plot
            self.ao_plots[channel].clear()
            self.ao_plots[channel].plot(ao_timestamps, ao_values)
            
            QApplication.processEvents()
        except Exception as e:
            print(f"Error updating plots: {str(e)}")
    
    def update_progress(self, value):
        self.progress_bar.setValue(value)

    def update_plot(self, channel, timestamps, values):
        self.ao_plots[channel].clear()
        self.ao_plots[channel].plot(timestamps * 1000, values)  # Convert timestamps to milliseconds for display
        self.ao_plots[channel].setLabel('bottom', 'Time', units='ms')
        self.ao_plots[channel].setXRange(0, timestamps[-1] * 1000)  # Ensure full range is visible
        QApplication.processEvents()
        
    def handle_thread_error(self, error_message):
        QMessageBox.critical(self, "Error", f"An error occurred: {error_message}")

    def output_finished(self, channel):
        QMessageBox.information(self, "Success", f"Waveform output completed for channel {channel}")
        
    def handle_error(self, error_info):
        QMessageBox.critical(self, "Error", str(error_info[1]))
                            
    def reset_ao_graph(self, channel):
        index = self.ao_channels.index(channel)
        self.ao_values[index] = np.array([])
        self.ao_timestamps[index] = np.array([])
        self.ao_plots[channel].clear()
    
    def read_ao_value(self, channel, index):
        try:
            with nidaqmx.Task() as task:
                task.ai_channels.add_ai_voltage_chan(channel,min_val=-10,max_val=10)
                value = task.read()
                self.ao_read_labels[index].setText(f"{value[0]:.2f}")
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error reading AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = DAQmxController()
    sys.exit(app.exec_())



In [ ]:
# measuring optical power

import sys
import numpy as np
import time
from functools import partial
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QHBoxLayout, QGridLayout, 
    QScrollArea, QComboBox, QLineEdit, QFileDialog, QMessageBox, QProgressDialog, QProgressBar
)
from PyQt5.QtCore import Qt, QThread, pyqtSignal, QRunnable, QThreadPool, QObject
import pyqtgraph as pg
import nidaqmx
from nidaqmx.constants import AcquisitionType, Edge, TerminalConfiguration
from nidaqmx.stream_writers import AnalogMultiChannelWriter
from PyQt5.QtCore import QTimer
import threading
import time
class AnalogInputTask(QThread):
    update_plot = pyqtSignal(dict)

    def __init__(self, channels, sample_rate, buffer_size, configs):
        super().__init__()
        self.channels = channels
        self.sample_rate = sample_rate
        self.buffer_size = buffer_size
        self.configs = configs
        self.task = None
        self.running = False

    def run(self):
        try:
            with nidaqmx.Task() as self.task:
                for channel, config in zip(self.channels, self.configs):
                    if config == 'RSE':
                        terminal_config = TerminalConfiguration.RSE
                    elif config == 'NRSE':
                        terminal_config = TerminalConfiguration.NRSE
                    elif config == 'DIFF':
                        terminal_config = TerminalConfiguration.DIFF
                    elif config == 'PSEUDO_DIFF':
                        terminal_config = TerminalConfiguration.PSEUDO_DIFF
                    else:
                        terminal_config = TerminalConfiguration.DEFAULT
                    
                    self.task.ai_channels.add_ai_voltage_chan(channel, terminal_config=terminal_config, min_val=-10, max_val=10)
                
                self.task.timing.cfg_samp_clk_timing(rate=self.sample_rate, sample_mode=AcquisitionType.CONTINUOUS)
                
                self.running = True
                self.start_time = time.time()
                self.task.start()
                
                while self.running:
                    data = self.task.read(number_of_samples_per_channel=self.buffer_size)
                    current_time = time.time()
                    data = np.array(data)
                    if data.ndim == 1:
                        data = data.reshape(1, -1)
                    timestamps = np.linspace(current_time - self.start_time - data.shape[1] / self.sample_rate,
                                             current_time - self.start_time,
                                             data.shape[1])
                    data_dict = {channel: data[i, :] for i, channel in enumerate(self.channels)}
                    data_dict['timestamps'] = timestamps
                    self.update_plot.emit(data_dict)
                    
                    time.sleep(0.01)  # Reduced delay

        except Exception as e:
            print(f"Error in continuous AI sampling: {str(e)}")

    def stop(self):
        print(f"Stopping AI task for channels: {', '.join(self.channels)}")
        self.running = False
        
class WorkerSignals(QObject):
    finished = pyqtSignal()
    error = pyqtSignal(tuple)
    result = pyqtSignal(object)

class Worker(QRunnable):
    def __init__(self, fn, *args, **kwargs):
        super().__init__()
        self.fn = fn
        self.args = args
        self.kwargs = kwargs
        self.signals = WorkerSignals()

    def run(self):
        try:
            result = self.fn(*self.args, **self.kwargs)
        except:
            exctype, value = sys.exc_info()[:2]
            self.signals.error.emit((exctype, value, traceback.format_exc()))
        else:
            self.signals.result.emit(result)
        finally:
            self.signals.finished.emit()

class OutputThread(QThread):
    update_plot = pyqtSignal(str, np.ndarray, np.ndarray)
    error_occurred = pyqtSignal(str)
    finished = pyqtSignal()

    def __init__(self, channel, values, period, iterations, sample_rate):
        super().__init__()
        self.channel = channel
        self.values = values
        self.period = period
        self.iterations = iterations
        self.sample_rate = sample_rate

    def run(self):
        try:
            single_period_samples = len(self.values)
            total_samples = single_period_samples * self.iterations
            waveform = np.tile(self.values, self.iterations)
            
            total_duration = self.period * self.iterations
            timestamps = np.linspace(0, total_duration, total_samples)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(self.channel)
                task.timing.cfg_samp_clk_timing(rate=self.sample_rate, sample_mode=AcquisitionType.FINITE, samps_per_chan=total_samples)
                
                task.write(waveform, auto_start=True)
                
                self.update_plot.emit(self.channel, timestamps, waveform)
                
                task.wait_until_done(timeout=total_duration + 5.0)
            
            self.finished.emit()
        except Exception as e:
            self.error_occurred.emit(str(e))

class DAQmxController(QWidget):
    def __init__(self):
        super().__init__()
        self.GAIN = 1  # Replace with your actual gain value
        self.GRADIENT = 6.6367e-10  # Replace with your actual gradient value in V/W
        self.Y_INTERCEPT = -4.0084e-10  # Replace with your actual y-intercept in watts
        
        self.threadpool = QThreadPool()
        self.ao_channels = ["Dev1/ao0", "Dev1/ao1", "Dev1/ao2", "Dev1/ao3"]
        self.ao_values = [np.array([]) for _ in range(len(self.ao_channels))]
        self.ao_timestamps = [np.array([]) for _ in range(len(self.ao_channels))]       
        self.ao_period_textboxes = []
        self.ao_iterations_textboxes = []
        self.ao_file_paths = [None] * len(self.ao_channels)
        self.progress_bar = QProgressBar(self)
        self.progress_bar.setVisible(False)
        self.uploaded_data = {channel: None for channel in self.ao_channels}
        self.run_buttons = []
        self.ao_preview_plots = {}
        
        self.ai_tasks = {}
        self.active_ai_channels = set()
        self.ai_channels = ["Dev1/ai0", "Dev1/ai1", "Dev1/ai2", "Dev1/ai3"]
        self.ai_tasks = {}
        self.ai_plots = {}
        self.ai_data = {channel: [[], []] for channel in self.ai_channels}  # (timestamps, values)
        self.ai_sampling_rates = [QLineEdit("10000") for _ in self.ai_channels]
        self.ai_curves = {}  # To store plot curves
        self.ai_config_dropdowns = {}
        self.ai_config_dropdowns = {}

        self.initUI()

    def initUI(self):
        main_layout = QVBoxLayout()
        # Create separate layouts for AO and AI controls
        ao_control_layout = QGridLayout()
        ai_control_layout = QGridLayout()
        
        # Create a horizontal layout to hold AO and AI control layouts side by side
        control_layout = QHBoxLayout()
        control_layout.addLayout(ao_control_layout)
        control_layout.addLayout(ai_control_layout)
        
        # Create a grid layout for all plots
        plot_layout = QGridLayout()
        plot_layout.setHorizontalSpacing(5)
        plot_layout.setVerticalSpacing(5)
        
        self.ao_labels = []
        self.ao_plots = {}
        self.ai_plots = {}

        # Set up AO controls and preview plots
        for i, ao_channel in enumerate(self.ao_channels):
            ao_label = QLabel(f"Analog Output {ao_channel}")
            ao_upload_button = QPushButton("Upload")
            ao_upload_button.clicked.connect(partial(self.upload_ao_values, ao_channel))

            ao_period_label = QLabel("Period (s):")
            ao_period_textbox = QLineEdit("0.005")
            ao_iterations_label = QLabel("Iterations:")
            ao_iterations_textbox = QLineEdit("1")

            ao_set_button = QPushButton("Set")
            ao_set_button.clicked.connect(partial(self.set_ao_settings, ao_channel))

            run_button = QPushButton("Run")
            run_button.clicked.connect(partial(self.run_ao_values, ao_channel))
            run_button.setEnabled(False)
            self.run_buttons.append(run_button)

            ao_reset_button = QPushButton("Reset")
            ao_reset_button.clicked.connect(partial(self.reset_ao_graph, ao_channel))

            ao_control_layout.addWidget(ao_label, i, 0)
            ao_control_layout.addWidget(ao_upload_button, i, 1)
            ao_control_layout.addWidget(ao_period_label, i, 2)
            ao_control_layout.addWidget(ao_period_textbox, i, 3)
            ao_control_layout.addWidget(ao_iterations_label, i, 4)
            ao_control_layout.addWidget(ao_iterations_textbox, i, 5)
            ao_control_layout.addWidget(ao_set_button, i, 6)
            ao_control_layout.addWidget(run_button, i, 7)
            ao_control_layout.addWidget(ao_reset_button, i, 8)

            self.ao_labels.append(ao_label)
            self.ao_period_textboxes.append(ao_period_textbox)
            self.ao_iterations_textboxes.append(ao_iterations_textbox)

            # Create and add AO preview plot
            preview_plot = pg.PlotWidget(title=f"Preview: {ao_channel}")
            preview_plot.setLabel('left', 'Voltage', units='V')
            preview_plot.setLabel('bottom', 'Sample')
            preview_plot.showGrid(x=True, y=True)
            preview_plot.setYRange(-10, 10, padding=0)
            preview_plot.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            preview_plot.setFixedSize(300, 200)
            self.ao_preview_plots[ao_channel] = preview_plot
            plot_layout.addWidget(preview_plot, i, 0)

            # Create and add AO plot
            plot_widget_ao = pg.PlotWidget(title=f"Analog Output Wave {ao_channel}")
            plot_widget_ao.setLabel('left', 'Voltage', units='V')
            plot_widget_ao.setLabel('bottom', 'Time', units='s')
            plot_widget_ao.showGrid(x=True, y=True)
            plot_widget_ao.setYRange(-10, 10, padding=0)
            plot_widget_ao.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            plot_widget_ao.setFixedSize(300, 200)
            self.ao_plots[ao_channel] = plot_widget_ao
            plot_layout.addWidget(plot_widget_ao, i, 1)

        # Set up AI controls and plots
        for i, ai_channel in enumerate(self.ai_channels):
            ai_label = QLabel(f"Analog Input {ai_channel}")
            ai_start_button = QPushButton("Start")
            ai_stop_button = QPushButton("Stop")
            ai_start_button.clicked.connect(partial(self.start_ai_task, ai_channel))
            ai_stop_button.clicked.connect(partial(self.stop_ai_task, ai_channel))

            ai_sampling_rate_label = QLabel("Sampling Rate:")
            ai_sampling_rate = QLineEdit("10000")  # Default to 10 kHz
            self.ai_sampling_rates.append(ai_sampling_rate)

            ai_control_layout.addWidget(ai_label, i, 0)
            ai_control_layout.addWidget(ai_sampling_rate_label, i, 1)
            ai_control_layout.addWidget(ai_sampling_rate, i, 2)
            ai_control_layout.addWidget(ai_start_button, i, 3)
            ai_control_layout.addWidget(ai_stop_button, i, 4)

            # Create and add AI plot
            plot_widget_ai = pg.PlotWidget(title=f"Analog Input {ai_channel}")
            plot_widget_ai.setLabel('left', 'Power', units='W')
            plot_widget_ai.setLabel('bottom', 'Time', units='s')
            plot_widget_ai.showGrid(x=True, y=True)
            plot_widget_ai.setYRange(0, 1)  
            plot_widget_ai.setXRange(0, 1)  # Set initial x-range to 1 second
            plot_widget_ai.setFixedSize(300, 200)  # Add this line
            self.ai_plots[ai_channel] = plot_widget_ai
            self.ai_curves[ai_channel] = plot_widget_ai.plot(pen='y')
            plot_layout.addWidget(plot_widget_ai, i, 2)
            
            # In the initUI method, where you set up AI controls
            ai_config_label = QLabel("Input Config:")
            ai_config_dropdown = QComboBox()
            ai_config_dropdown.addItems(["RSE", "NRSE", "Differential", "PSEUDO_DIFF"])
            ai_config_dropdown.setCurrentText("RSE")  # Set default to RSE
            ai_config_dropdown.currentTextChanged.connect(partial(self.change_ai_config, ai_channel))

            ai_control_layout.addWidget(ai_config_label, i, 5)
            ai_control_layout.addWidget(ai_config_dropdown, i, 6)
            
            ai_set_button = QPushButton("Set")
            ai_set_button.clicked.connect(partial(self.set_ai_config, ai_channel))
            ai_control_layout.addWidget(ai_set_button, i, 7)
            
            ai_reset_button = QPushButton("Reset")
            ai_reset_button.clicked.connect(partial(self.reset_ai_graph, ai_channel))
            ai_control_layout.addWidget(ai_reset_button, i, 8)

            # Store the dropdown in a dictionary for later access
            self.ai_config_dropdowns[ai_channel] = ai_config_dropdown

        # Create scroll areas for controls and plots
        control_widget = QWidget()
        control_widget.setLayout(control_layout)
        control_scroll_area = QScrollArea()
        control_scroll_area.setWidget(control_widget)
        control_scroll_area.setWidgetResizable(True)
        control_scroll_area.setFixedHeight(200)

        plot_widget = QWidget()
        plot_widget.setLayout(plot_layout)
        plot_scroll_area = QScrollArea()
        plot_scroll_area.setWidget(plot_widget)
        plot_scroll_area.setWidgetResizable(True)

        main_layout.addWidget(control_scroll_area)
        main_layout.addWidget(plot_scroll_area)
        
        self.setLayout(main_layout)
        self.setWindowTitle('DAQmx Analog IO Controller')
        self.showMaximized()
    
    def voltage_to_power(self, voltage):
        power=((voltage / (self.GAIN * self.GRADIENT ))+ (self.Y_INTERCEPT/ (self.GAIN * self.GRADIENT )))
        return power

    def change_ai_config(self, channel, config):
        if channel in self.ai_tasks:
            self.stop_ai_task(channel)
        self.start_ai_task(channel)

    def reset_ai_graph(self, channel):
        self.ai_data[channel] = [[], []]
        self.ai_curves[channel].setData([], [])
        self.ai_plots[channel].setYRange(-10, 10)
        self.ai_plots[channel].setXRange(0, 6)
    
    def set_ai_config(self, channel):
        if channel in self.ai_tasks:
            self.stop_ai_task(channel)
        self.start_ai_task(channel)
    
    def update_all_ai_tasks(self):
        if not self.ai_tasks:
            print("No active AI tasks to update.")
            return

        max_rate = self.calculate_max_sampling_rate()
        for channel, task in self.ai_tasks.items():
            new_rate = min(float(self.ai_sampling_rates[self.ai_channels.index(channel)].text()), max_rate)
            task.update_sample_rate(new_rate)
            print(f"Updated {channel} to sample rate {new_rate}")
                
    def start_ai_task(self, channel):
        print(f"Attempting to start AI task for {channel}")
        
        if channel in self.active_ai_channels:
            print(f"Channel {channel} is already running.")
            return

        self.active_ai_channels.add(channel)
        
        # Update or create the task for this channel
        max_rate = self.calculate_max_sampling_rate()
        sample_rate = min(max_rate, float(self.ai_sampling_rates[self.ai_channels.index(channel)].text()))
        buffer_size = int(sample_rate * 0.1)  # 0.1 seconds of data
        config = self.ai_config_dropdowns[channel].currentText()

        if not self.ai_tasks:
            # If no task exists, create a new one
            task = AnalogInputTask([channel], sample_rate, buffer_size, [config])
            task.update_plot.connect(self.update_ai_plots)
            task.start()
            self.ai_tasks[channel] = task
        else:
            # If a task exists, add this channel to it
            existing_task = next(iter(self.ai_tasks.values()))
            existing_task.channels.append(channel)
            existing_task.configs.append(config)
            existing_task.stop()
            existing_task.wait()
            
            # Recreate the task with the updated channels
            new_task = AnalogInputTask(existing_task.channels, sample_rate, buffer_size, existing_task.configs)
            new_task.update_plot.connect(self.update_ai_plots)
            new_task.start()
            
            # Update all channel references to the new task
            for ch in existing_task.channels:
                self.ai_tasks[ch] = new_task

        print(f"Started AI task for channel: {channel} with sample rate {sample_rate}")

    def stop_ai_task(self, channel):
        if channel not in self.active_ai_channels:
            print(f"Channel {channel} is not running.")
            return

        self.active_ai_channels.remove(channel)
        
        if channel in self.ai_tasks:
            task = self.ai_tasks[channel]
            task.channels.remove(channel)
            task.configs.pop(task.channels.index(channel))
            
            if not task.channels:
                # If no channels left, stop and remove the task
                task.stop()
                task.wait()
                self.ai_tasks.clear()
            else:
                # Recreate the task with the remaining channels
                sample_rate = task.sample_rate
                buffer_size = task.buffer_size
                new_task = AnalogInputTask(task.channels, sample_rate, buffer_size, task.configs)
                new_task.update_plot.connect(self.update_ai_plots)
                new_task.start()
                
                # Update all channel references to the new task
                for ch in task.channels:
                    self.ai_tasks[ch] = new_task
                
                # Stop the old task
                task.stop()
                task.wait()

            del self.ai_tasks[channel]
        
        print(f"Stopped AI task for channel: {channel}")

    def update_ai_plots(self, data_dict):
        timestamps = data_dict['timestamps']
        for channel in self.active_ai_channels:
            if channel in data_dict:
                voltages = data_dict[channel]
                
                # Ensure timestamps and voltages are arrays
                if not isinstance(timestamps, np.ndarray):
                    timestamps = np.array([timestamps])
                if not isinstance(voltages, np.ndarray):
                    voltages = np.array([voltages])
                
                # Convert voltages to power
                powers = self.voltage_to_power(voltages)
                
                self.ai_data[channel][0].extend(timestamps)
                self.ai_data[channel][1].extend(powers)
                
                # Keep only the last 60000 points (6 seconds at 10kHz)
                if len(self.ai_data[channel][0]) > 60000:
                    self.ai_data[channel][0] = self.ai_data[channel][0][-60000:]
                    self.ai_data[channel][1] = self.ai_data[channel][1][-60000:]
                
                self.ai_curves[channel].setData(self.ai_data[channel][0], self.ai_data[channel][1])
                
                # Update axis ranges
                if len(self.ai_data[channel][0]) > 0:  # Check if we have any data
                    x_min = max(0, self.ai_data[channel][0][-1] - 6)
                    x_max = self.ai_data[channel][0][-1]
                    self.ai_plots[channel].setXRange(x_min, x_max)
                    y_min, y_max = min(self.ai_data[channel][1]), max(self.ai_data[channel][1])
                    y_range = y_max - y_min
                    self.ai_plots[channel].setYRange(y_min - 0.1 * y_range, y_max + 0.1 * y_range)
                
                print(f"Updating plot for {channel}: min={np.min(powers):.3f}W, max={np.max(powers):.3f}W, len={len(self.ai_data[channel][0])}")

    def calculate_max_sampling_rate(self):
        active_channels = len(self.active_ai_channels)
        if active_channels == 0:
            return 600000  # Return max rate if no channels are active
        elif active_channels == 1:
            return 600000  # Max for single channel
        else:
            return min(500000 // active_channels, 600000)  # Respect multi-channel limit
        
    def set_ao_value(self, channel, textbox):
        try:
            value = float(textbox.text())
            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.write(value)
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error setting AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def sweep_ao_voltage(self, channel, start_voltage, end_voltage, voltage_step, iterations):
        try:
            start = float(start_voltage.text())
            end = float(end_voltage.text())
            step = float(voltage_step.text())
            num_iterations = int(iterations.text())

            if start >= end:
                raise ValueError("Start voltage must be less than end voltage.")
            if step <= 0:
                raise ValueError("Voltage step must be greater than zero.")
            if num_iterations <= 0:
                raise ValueError("Number of iterations must be greater than zero.")

            voltages = np.arange(start, end + step, step)
            waveform = np.tile(voltages, num_iterations)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.timing.cfg_samp_clk_timing(rate=1000, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                task.write(waveform, auto_start=True)

                index = self.ao_channels.index(channel)
                timestamps = np.linspace(0, len(waveform) / 1000, len(waveform))
                for i in range(len(waveform)):
                    self.ao_values[index].append(waveform[i])
                    self.ao_timestamps[index].append(timestamps[i])
                    self.ao_plots[channel].clear()
                    self.ao_plots[channel].plot(self.ao_timestamps[index], self.ao_values[index])
                    QApplication.processEvents()

                task.wait_until_done()
                task.stop()

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error sweeping AO voltage: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def set_ao_settings(self, channel):
        try:
            index = self.ao_channels.index(channel)
            period = float(self.ao_period_textboxes[index].text())
            iterations = int(self.ao_iterations_textboxes[index].text())

            if period <= 0:
                raise ValueError("Period must be greater than zero.")
            if iterations <= 0:
                raise ValueError("Iterations must be greater than zero.")

            if self.uploaded_data[channel] is None:
                raise ValueError("No data uploaded for this channel.")

            values = self.uploaded_data[channel]
            samples = len(values)
            t = np.linspace(0, period * iterations, samples * iterations)
            preview_waveform = np.tile(values, iterations)

            self.ao_preview_plots[channel].clear()
            self.ao_preview_plots[channel].plot(t, preview_waveform)
            self.ao_preview_plots[channel].setLabel('bottom', 'Time', units='s')

            QMessageBox.information(self, "Settings Applied", f"Settings applied for {channel}. Check the preview graph.")

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Error setting AO values: {str(e)}")
            
    def upload_ao_values(self, channel):
        try:
            file_dialog = QFileDialog()
            file_path, _ = file_dialog.getOpenFileName(self, "Open Text File", "", "Text Files (*.txt)")
            if file_path:
                worker = Worker(self._load_file, file_path, channel)
                worker.signals.result.connect(self.update_preview)
                worker.signals.error.connect(self.handle_error)
                self.threadpool.start(worker)
        except Exception as e:
            self.handle_error(("Error", e, ""))

    def _load_file(self, file_path, channel):
        with open(file_path, "r") as file:
            values = [float(line.strip()) for line in file.readlines()]
        return channel, values

    def update_preview(self, result):
        channel, values = result
        self.uploaded_data[channel] = values
        index = self.ao_channels.index(channel)
        
        self.ao_preview_plots[channel].clear()
        self.ao_preview_plots[channel].plot(range(len(values)), values)
        
        self.run_buttons[index].setEnabled(True)
    
    def closeEvent(self, event):
        for channel in list(self.ai_tasks.keys()):
            self.stop_ai_task(channel)
        for task in nidaqmx.system.System().tasks:
            try:
                task.close()
            except:
                pass
        event.accept()

    def run_ao_values(self, channel):
        if self.uploaded_data[channel] is None:
            QMessageBox.warning(self, "Warning", "No data uploaded for this channel.")
            return
        
        index = self.ao_channels.index(channel)
        values = self.uploaded_data[channel]
        period = float(self.ao_period_textboxes[index].text())
        iterations = int(self.ao_iterations_textboxes[index].text())

        sample_rate = len(values) / period
        samples_per_channel = len(values) * iterations

        waveform = np.tile(values, iterations)

        try:
            with nidaqmx.Task() as ao_task:
                ao_task.ao_channels.add_ao_voltage_chan(channel)
                ao_task.timing.cfg_samp_clk_timing(rate=sample_rate,
                                                sample_mode=AcquisitionType.FINITE,
                                                samps_per_chan=samples_per_channel)

                ao_writer = AnalogMultiChannelWriter(ao_task.out_stream)

                ao_writer.write_many_sample(waveform.reshape(1, -1))

                start_time = time.perf_counter()
                ao_task.start()

                ao_task.wait_until_done(timeout=period * iterations + 5.0)

                end_time = time.perf_counter()

            timestamps = np.linspace(0, period * iterations, samples_per_channel)

            self.update_plot(channel, timestamps, waveform)

            expected_duration = period * iterations
            actual_duration = end_time - start_time
            timing_error = abs(actual_duration - expected_duration)

            info_message = (f"Waveform output completed for channel {channel}\n"
                            f"Expected duration: {expected_duration:.6f}s\n"
                            f"Actual duration: {actual_duration:.6f}s\n"
                            f"Timing error: {timing_error:.6f}s")
            
            QMessageBox.information(self, "Success", info_message)

        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error during AO operation: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")
        
    def update_plots(self, channel, ao_timestamps, ao_values):
        try:
            # Update AO plot
            self.ao_plots[channel].clear()
            self.ao_plots[channel].plot(ao_timestamps, ao_values)
            
            QApplication.processEvents()
        except Exception as e:
            print(f"Error updating plots: {str(e)}")
    
    def update_progress(self, value):
        self.progress_bar.setValue(value)

    def update_plot(self, channel, timestamps, values):
        self.ao_plots[channel].clear()
        self.ao_plots[channel].plot(timestamps * 1000, values)  # Convert timestamps to milliseconds for display
        self.ao_plots[channel].setLabel('bottom', 'Time', units='ms')
        self.ao_plots[channel].setXRange(0, timestamps[-1] * 1000)  # Ensure full range is visible
        QApplication.processEvents()
        
    def handle_thread_error(self, error_message):
        QMessageBox.critical(self, "Error", f"An error occurred: {error_message}")

    def output_finished(self, channel):
        QMessageBox.information(self, "Success", f"Waveform output completed for channel {channel}")
        
    def handle_error(self, error_info):
        QMessageBox.critical(self, "Error", str(error_info[1]))
                            
    def reset_ao_graph(self, channel):
        index = self.ao_channels.index(channel)
        self.ao_values[index] = np.array([])
        self.ao_timestamps[index] = np.array([])
        self.ao_plots[channel].clear()
    
    def read_ao_value(self, channel, index):
        try:
            with nidaqmx.Task() as task:
                task.ai_channels.add_ai_voltage_chan(channel,min_val=-10,max_val=10)
                value = task.read()
                self.ao_read_labels[index].setText(f"{value[0]:.2f}")
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error reading AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = DAQmxController()
    sys.exit(app.exec_())



In [ ]:
#Deconstruction "deconstructed_image_data.txt"
import numpy as np
from PIL import Image

def deconstruct_image(file_path, output_file_path):
    # Load the image
    img = Image.open(file_path).convert('L')  # Convert to grayscale
    gray_scale_img = np.array(img)

    # Ensure the image is 426x640
    Height = 99
    Width = 99
    if gray_scale_img.shape != (Height, Width):  # Height x Width
        raise ValueError(f"Image size must be {Height}x{Width}, but got {gray_scale_img.shape}")

    # Print original image shape
    print(f"Original image shape: {gray_scale_img.shape}")

    # Flatten the image
    vector = gray_scale_img.flatten()

    # Calculate the size of each row (must be divisible by 3)
    total_pixels = Height * Width
    row_size = total_pixels // 3

    # Reshape into 3 rows
    three_rows = vector.reshape(3, row_size)

    print(f"Three rows shape: {three_rows.shape}")

    # Divide the values by 100
    three_rows_divided = three_rows / 100.0

    # Save the three rows to a text file
    np.savetxt(output_file_path, three_rows_divided.T, fmt='%.2f', delimiter='\t')
    print(f"Deconstructed image data saved to {output_file_path}")

if __name__ == "__main__":
    file_path = r"C:\Users\bovta\Desktop\Aaron (Intern)\Aaron (Intern)\VS code Stuff\send image\sizephoto.jpg"
    output_file_path = r"C:\Users\bovta\Desktop\Aaron (Intern)\Aaron (Intern)\VS code Stuff\send image\deconstructed_image_data.txt"
    
    deconstruct_image(file_path, output_file_path)

In [ ]:
#Deconstruction for 255 values "deconstructed_array_data.txt"
import numpy as np

def create_and_deconstruct_array(output_file_path):
    # Create a 1D array with values from 0 to 255
    vector = np.arange(256, dtype=np.float64)

    # Print original array shape
    print(f"Original array shape: {vector.shape}")

    # Calculate the size of each row (must be divisible by 3)
    total_elements = len(vector)
    row_size = total_elements // 3

    # If the total number of elements is not divisible by 3, we'll pad the array
    if total_elements % 3 != 0:
        padding = 3 - (total_elements % 3)
        vector = np.pad(vector, (0, padding), mode='constant', constant_values=0)
        total_elements = len(vector)
        row_size = total_elements // 3

    # Reshape into 3 rows
    three_rows = vector.reshape(3, row_size)

    print(f"Three rows shape: {three_rows.shape}")

    # Divide the values by 100
    three_rows_divided = three_rows / 100.0

    # Save the three rows to a text file
    np.savetxt(output_file_path, three_rows_divided.T, fmt='%.2f', delimiter='\t')
    print(f"Deconstructed array data saved to {output_file_path}")

if __name__ == "__main__":
    output_file_path = r"C:\Users\bovta\Desktop\Aaron (Intern)\Aaron (Intern)\VS code Stuff\send image\deconstructed_array_data.txt"
    
    create_and_deconstruct_array(output_file_path)

In [ ]:
#Reconstruction
import numpy as np
import matplotlib.pyplot as plt

def reconstruct_image(input_file_path, output_image_path):
    # Load the data from the text file
    three_rows_divided = np.loadtxt(input_file_path, delimiter='\t').T

    # Multiply by 100 and convert back to uint8
    three_rows = (three_rows_divided * 100).astype(np.uint8)

    # Reconstruct the image
    Height = 99
    Width = 99
    reconstructed_vector = three_rows.flatten()
    reconstructed_img = reconstructed_vector.reshape((Height, Width))

    print(f"Reconstructed image shape: {reconstructed_img.shape}")

    # Save the reconstructed image
    plt.imsave(output_image_path, reconstructed_img, cmap='gray')
    print(f"Reconstructed image saved to {output_image_path}")

    # Display the reconstructed image
    plt.imshow(reconstructed_img, cmap='gray')
    plt.title("Reconstructed Image")
    plt.show()

if __name__ == "__main__":
    input_file_path = r"C:\Users\bovta\Desktop\Aaron (Intern)\Aaron (Intern)\VS code Stuff\send image\Read_Pd.txt"
    output_image_path = r"C:\Users\bovta\Desktop\Aaron (Intern)\Aaron (Intern)\VS code Stuff\send image\reconstructed_image.png"
    
    reconstruct_image(input_file_path, output_image_path)

In [ ]:
# Send read

import nidaqmx
import numpy as np
from nidaqmx.constants import AcquisitionType, TerminalConfiguration
from nidaqmx.stream_readers import AnalogMultiChannelReader
from nidaqmx.stream_writers import AnalogMultiChannelWriter

def prepare_multichannel_data(file_path, num_channels):
    data = np.loadtxt(file_path, delimiter='\t')
    if data.shape[1] < num_channels:
        padding = np.zeros((data.shape[0], num_channels - data.shape[1]))
        data = np.hstack((data, padding))
    elif data.shape[1] > num_channels:
        data = data[:, :num_channels]
    return data.T

def read_lookup_table(file_path):
    data = np.loadtxt(file_path, skiprows=1)
    input_values = data[:, 0]
    output_values = data[:, 1]
    return input_values, output_values

def find_closest_value(value, array):
    return array[np.argmin(np.abs(array - value))]

# Constants
SAMPLE_RATE = 100
NUM_CHANNELS = 3

# Prepare write data from file
file_path = 'deconstructed_image_data.txt'
write_data = prepare_multichannel_data(file_path, NUM_CHANNELS)
SAMPLES_PER_CHANNEL = write_data.shape[1]

# Read the look-up table
input_lut, output_lut = read_lookup_table('Look-up table1.txt')

# Process write_data through look-up table
After_LUT_v = np.zeros_like(write_data)
for channel in range(NUM_CHANNELS):
    for i in range(SAMPLES_PER_CHANNEL):
        closest_output = find_closest_value(write_data[channel][i], output_lut)
        corresponding_input = input_lut[np.where(output_lut == closest_output)[0][0]]
        After_LUT_v[channel][i] = corresponding_input

# Ensure After_LUT_v is C-contiguous
After_LUT_v = np.ascontiguousarray(After_LUT_v)

with nidaqmx.Task() as read_task, nidaqmx.Task() as write_task:
    # Setup channels
    for i in range(NUM_CHANNELS):
        read_task.ai_channels.add_ai_voltage_chan(f"Dev1/ai{i}", terminal_config=TerminalConfiguration.RSE, min_val=-10.0, max_val=10.0)
        write_task.ao_channels.add_ao_voltage_chan(f"Dev1/ao{i}", min_val=-10.0, max_val=10.0)

    # Configure timing for both tasks
    write_task.timing.cfg_samp_clk_timing(rate=SAMPLE_RATE, sample_mode=AcquisitionType.FINITE, samps_per_chan=SAMPLES_PER_CHANNEL)
    read_task.timing.cfg_samp_clk_timing(rate=SAMPLE_RATE, source="ao/SampleClock", sample_mode=AcquisitionType.FINITE, samps_per_chan=SAMPLES_PER_CHANNEL)

    # Configure start trigger for read task
    read_task.triggers.start_trigger.cfg_dig_edge_start_trig("/Dev1/ao/StartTrigger")
    
    read_task.timing.delay_from_samp_clk_delay_units = nidaqmx.constants.DigitalWidthUnits.SECONDS
    read_task.timing.delay_from_samp_clk_delay = 0.5/SAMPLE_RATE # update this with the value in seconds to delay

    # Create reader and writer
    reader = AnalogMultiChannelReader(read_task.in_stream)
    writer = AnalogMultiChannelWriter(write_task.out_stream)

    # Write After_LUT_v data
    writer.write_many_sample(After_LUT_v)
    
    # Start the tasks
    read_task.start()
    write_task.start()
    
    # Read the acquired data
    read_data = np.zeros((NUM_CHANNELS, SAMPLES_PER_CHANNEL), dtype=np.float64)
    reader.read_many_sample(read_data, number_of_samples_per_channel=SAMPLES_PER_CHANNEL, timeout=SAMPLES_PER_CHANNEL/SAMPLE_RATE + 5.0)
    
    # Wait for the tasks to complete
    write_task.wait_until_done()
    read_task.wait_until_done()
    
    # Stop the tasks
    write_task.stop()
    read_task.stop()

# Save After_LUT_v values to file
np.savetxt('After_LUT_v.txt', After_LUT_v.T, delimiter='\t', fmt='%.6f')

# Save Read_Pd (read_data) values to file
np.savetxt('Read_Pd.txt', read_data.T, delimiter='\t', fmt='%.6f')

print(f"Data acquisition and processing complete. Results written to 'After_LUT_v.txt' and 'Read_Pd.txt'.")
print(f"Number of samples per channel: {SAMPLES_PER_CHANNEL}")

In [ ]:
# Sweep voltage generator

# Generate sweeping voltage
import numpy as np

# Generate the voltage sweep
start_voltage = 0
end_voltage = 5
num_samples = 1000

voltages = np.linspace(start_voltage, end_voltage, num_samples)

# Create a file and write the voltages
filename = "voltage_sweep.txt"

with open(filename, 'w') as file:
    for voltage in voltages:
        file.write(f"{voltage:.6f}\n")

print(f"Voltage sweep from {start_voltage}V to {end_voltage}V with {num_samples} samples has been written to {filename}")

# Optional: Print the first few and last few values to verify
print("\nFirst 5 values:")
print(voltages[:5])
print("\nLast 5 values:")
print(voltages[-5:])

In [ ]:
# UI for voltage sweep

#Single channel
import sys
import numpy as np
import time
from functools import partial
from PyQt5.QtWidgets import (
    QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QHBoxLayout, QGridLayout, 
    QScrollArea, QComboBox, QLineEdit, QFileDialog, QMessageBox, QProgressDialog, QProgressBar
)
from PyQt5.QtCore import Qt, QThread, pyqtSignal, QRunnable, QThreadPool, QObject
import pyqtgraph as pg
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, AcquisitionType, Edge
from nidaqmx.stream_writers import AnalogMultiChannelWriter
from nidaqmx.stream_readers import AnalogMultiChannelReader


class WorkerSignals(QObject):
    finished = pyqtSignal()
    error = pyqtSignal(tuple)
    result = pyqtSignal(object)

class Worker(QRunnable):
    def __init__(self, fn, *args, **kwargs):
        super().__init__()
        self.fn = fn
        self.args = args
        self.kwargs = kwargs
        self.signals = WorkerSignals()

    def run(self):
        try:
            result = self.fn(*self.args, **self.kwargs)
        except:
            exctype, value = sys.exc_info()[:2]
            self.signals.error.emit((exctype, value, traceback.format_exc()))
        else:
            self.signals.result.emit(result)
        finally:
            self.signals.finished.emit()

class OutputThread(QThread):
    update_plot = pyqtSignal(str, np.ndarray, np.ndarray)
    error_occurred = pyqtSignal(str)
    finished = pyqtSignal()

    def __init__(self, channel, values, period, iterations, sample_rate):
        super().__init__()
        self.channel = channel
        self.values = values
        self.period = period
        self.iterations = iterations
        self.sample_rate = sample_rate

    def run(self):
        try:
            single_period_samples = len(self.values)
            total_samples = single_period_samples * self.iterations
            waveform = np.tile(self.values, self.iterations)
            
            # Calculate total duration based on period and iterations
            total_duration = self.period * self.iterations
            timestamps = np.linspace(0, total_duration, total_samples)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(self.channel)
                task.timing.cfg_samp_clk_timing(rate=self.sample_rate, sample_mode=AcquisitionType.FINITE, samps_per_chan=total_samples)
                
                task.write(waveform, auto_start=True)
                
                self.update_plot.emit(self.channel, timestamps, waveform)
                
                task.wait_until_done(timeout=total_duration + 5.0)
            
            self.finished.emit()
        except Exception as e:
            self.error_occurred.emit(str(e))
            
class DataProcessingThread(QThread):
    update_progress = pyqtSignal(int)
    update_plot = pyqtSignal(str, np.ndarray, np.ndarray, np.ndarray, np.ndarray)
    error_occurred = pyqtSignal(str)
    
    def __init__(self, channel, waveform, total_duration, ai_task, ao_task):
        super().__init__()
        self.channel = channel
        self.waveform = waveform
        self.total_duration = total_duration
        self.stop_event = threading.Event()

    def run(self):
        try:
            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO task
                ao_task.ao_channels.add_ao_voltage_chan(self.channel)
                ao_task.timing.cfg_samp_clk_timing(rate=len(self.waveform)/self.total_duration, 
                                                   sample_mode=AcquisitionType.FINITE, 
                                                   samps_per_chan=len(self.waveform))

                # Configure AI task
                ai_channel = f"Dev1/ai{self.channel[-1]}"  # Assumes channel format like "Dev1/ao0"
                ai_task.ai_channels.add_ai_voltage_chan(ai_channel)
                ai_task.timing.cfg_samp_clk_timing(rate=len(self.waveform)/self.total_duration, 
                                                   sample_mode=AcquisitionType.FINITE, 
                                                   samps_per_chan=len(self.waveform))

                # Set up synchronization
                ai_task.triggers.start_trigger.cfg_dig_edge_start_trig(f"/{self.channel}/StartTrigger")

                writer = AnalogMultiChannelWriter(ao_task.out_stream)
                reader = AnalogMultiChannelReader(ai_task.in_stream)

                writer.write_many_sample(np.array([self.waveform]))
                
                ao_task.start()
                ai_task.start()

                ao_data = self.waveform
                ai_data = np.zeros((1, len(self.waveform)))
                timestamps = np.linspace(0, self.total_duration, len(self.waveform))

                batch_size = 1000
                for i in range(0, len(self.waveform), batch_size):
                    if self.stop_event.is_set():
                        break
                    end = min(i + batch_size, len(self.waveform))
                    reader.read_many_sample(
                        ai_data[:, i:end], 
                        number_of_samples_per_channel=end - i,
                        timeout=5.0
                    )
                    progress = int((i / len(self.waveform)) * 100)
                    self.update_progress.emit(progress)
                    
                    self.update_plot.emit(self.channel, timestamps[:end], ao_data[:end], timestamps[:end], ai_data[0, :end])

                ao_task.wait_until_done(timeout=self.total_duration + 5.0)
                ai_task.wait_until_done(timeout=self.total_duration + 5.0)
                
                self.update_plot.emit(self.channel, timestamps, ao_data, timestamps, ai_data[0])
                self.update_progress.emit(100)

        except Exception as e:
            self.error_occurred.emit(str(e))
        finally:
            self.stop_event.set()

    def stop(self):
        self.stop_event.set()

class DAQmxController(QWidget):
    def __init__(self):
        super().__init__()
        self.threadpool = QThreadPool()

        self.ao_channels = ["Dev1/ao0", "Dev1/ao1", "Dev1/ao2", "Dev1/ao3"]
        self.ai_channels = ["Dev1/ai0", "Dev1/ai1", "Dev1/ai2", "Dev1/ai3"]
        self.terminal_configs = ["RSE", "Differential", "Pseudodifferential"]

        self.ao_values = [np.array([]) for _ in range(len(self.ao_channels))]
        self.ai_values = [np.array([]) for _ in range(len(self.ai_channels))]
        self.ao_timestamps = [np.array([]) for _ in range(len(self.ao_channels))]
        self.ai_timestamps = [np.array([]) for _ in range(len(self.ai_channels))]
        
        self.ao_period_textboxes = []
        self.ao_iterations_textboxes = []
        self.ao_file_paths = [None] * len(self.ao_channels)
        self.progress_bar = QProgressBar(self)
        self.progress_bar.setVisible(False)
        self.uploaded_data = {channel: None for channel in self.ao_channels}
        self.run_buttons = []
        self.ao_preview_plots = {}  # New dictionary for preview plots
        
        self.initUI()

    def initUI(self):
        main_layout = QVBoxLayout()
        control_layout = QGridLayout()
        control_layout.setHorizontalSpacing(2)
        control_layout.setVerticalSpacing(2)

        # Create a grid layout for the plots
        plot_layout = QGridLayout()
        plot_layout.setHorizontalSpacing(5)
        plot_layout.setVerticalSpacing(5)
        
        self.ao_labels = []
        self.ai_labels = []
        self.ao_plots = {}
        self.ai_plots = {}
        self.ai_terminal_configs = []
        self.ai_sampling_rates = []
        self.ai_min_voltages = []
        self.ai_max_voltages = []

        # Add this after creating other AI channel controls
        for i, ai_channel in enumerate(self.ai_channels):
            ai_save_button = QPushButton("Save AI Data")
            ai_save_button.clicked.connect(partial(self.save_ai_data, ai_channel))
            control_layout.addWidget(ai_save_button, i + len(self.ao_channels), 10)
            
        # Add "Run" buttons for each channel
        for i, ao_channel in enumerate(self.ao_channels):
            run_button = QPushButton("Run")
            run_button.clicked.connect(partial(self.run_ao_values, ao_channel))
            run_button.setEnabled(False)
            self.run_buttons.append(run_button)
            control_layout.addWidget(run_button, i, 9)
            
        # Create textboxes, labels, upload buttons, period, iterations fields, and set buttons for analog output channels
        for i, ao_channel in enumerate(self.ao_channels):
            ao_label = QLabel(f"Analog Output {ao_channel}")
            ao_upload_button = QPushButton("Upload")
            ao_upload_button.clicked.connect(partial(self.upload_ao_values, ao_channel))

            ao_period_label = QLabel("Period (s):")
            ao_period_textbox = QLineEdit("0.005")  # Default period of 0.005 seconds (200 Hz)
            ao_iterations_label = QLabel("Iterations:")
            ao_iterations_textbox = QLineEdit("1")  # Default iterations of 1

            ao_set_button = QPushButton("Set")
            ao_set_button.clicked.connect(partial(self.set_ao_settings, ao_channel))

            control_layout.addWidget(ao_label, i, 0)
            control_layout.addWidget(ao_upload_button, i, 1)
            control_layout.addWidget(ao_period_label, i, 2)
            control_layout.addWidget(ao_period_textbox, i, 3)
            control_layout.addWidget(ao_iterations_label, i, 4)
            control_layout.addWidget(ao_iterations_textbox, i, 5)
            control_layout.addWidget(ao_set_button, i, 6)

            self.ao_labels.append(ao_label)
            self.ao_period_textboxes.append(ao_period_textbox)
            self.ao_iterations_textboxes.append(ao_iterations_textbox)

        # Create labels, read buttons, terminal configuration, sampling rate, and voltage range input fields for analog input channels
        for i, ai_channel in enumerate(self.ai_channels):
            ai_label = QLabel(f"Analog Input {ai_channel}")
            ai_value_label = QLabel("0.0")

            ai_terminal_config = QComboBox()
            ai_terminal_config.addItems(self.terminal_configs)
            self.ai_terminal_configs.append(ai_terminal_config)

            ai_sampling_rate = QLineEdit("1000")
            self.ai_sampling_rates.append(ai_sampling_rate)

            ai_min_voltage = QLineEdit("-10")
            ai_max_voltage = QLineEdit("10")
            self.ai_min_voltages.append(ai_min_voltage)
            self.ai_max_voltages.append(ai_max_voltage)

            control_layout.addWidget(ai_label, i + len(self.ao_channels), 0)
            control_layout.addWidget(ai_value_label, i + len(self.ao_channels), 1)
            control_layout.addWidget(QLabel("Terminal Config:"), i + len(self.ao_channels), 2)
            control_layout.addWidget(ai_terminal_config, i + len(self.ao_channels), 3)
            control_layout.addWidget(QLabel("Sampling Rate:"), i + len(self.ao_channels), 4)
            control_layout.addWidget(ai_sampling_rate, i + len(self.ao_channels), 5)
            control_layout.addWidget(QLabel("Min Voltage:"), i + len(self.ao_channels), 6)
            control_layout.addWidget(ai_min_voltage, i + len(self.ao_channels), 7)
            control_layout.addWidget(QLabel("Max Voltage:"), i + len(self.ao_channels), 8)
            control_layout.addWidget(ai_max_voltage, i + len(self.ao_channels), 9)

            self.ai_labels.append(ai_value_label)

            # Start reading AI values
            self.read_ai_value(ai_channel, ai_value_label, i)
        
        control_widget = QWidget()
        control_widget.setLayout(control_layout)

        scroll_area = QScrollArea()
        scroll_area.setWidget(control_widget)
        scroll_area.setWidgetResizable(True)
        scroll_area.setFixedHeight(400)

        main_layout.addWidget(scroll_area)

        # Create plot widgets for each analog output channel
        for i, ao_channel in enumerate(self.ao_channels):
            plot_widget_ao = pg.PlotWidget(title=f"Analog Output Wave {ao_channel}")
            plot_widget_ao.setLabel('left', 'Voltage', units='V')
            plot_widget_ao.setLabel('bottom', 'Time', units='s')
            plot_widget_ao.showGrid(x=True, y=True)
            plot_widget_ao.setYRange(-10, 10, padding=0)
            plot_widget_ao.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            plot_widget_ao.setFixedSize(300, 200)
            self.ao_plots[ao_channel] = plot_widget_ao
            plot_layout.addWidget(plot_widget_ao, i, 0)
            
            # Create separate preview plot widgets for each analog output channel
            preview_plot = pg.PlotWidget(title=f"Preview: {ao_channel}")
            preview_plot.setLabel('left', 'Voltage', units='V')
            preview_plot.setLabel('bottom', 'Sample')
            preview_plot.showGrid(x=True, y=True)
            preview_plot.setYRange(-10, 10, padding=0)
            preview_plot.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            preview_plot.setFixedSize(300, 200)
            self.ao_preview_plots[ao_channel] = preview_plot
            plot_layout.addWidget(preview_plot, i,1)

        for i, ai_channel in enumerate(self.ai_channels):
            plot_widget_ai = pg.PlotWidget(title=f"Analog Input Wave {ai_channel}")
            plot_widget_ai.setLabel('left', 'Voltage', units='V')
            plot_widget_ai.setLabel('bottom', 'Time', units='s')
            plot_widget_ai.showGrid(x=True, y=True)
            plot_widget_ai.setYRange(-10, 10, padding=0)
            plot_widget_ai.getAxis('left').setTicks([[(v, str(v)) for v in range(-10, 11, 2)]])
            plot_widget_ai.setFixedSize(300, 200)
            self.ai_plots[ai_channel] = plot_widget_ai
            plot_layout.addWidget(plot_widget_ai,i,2)

        plot_widget = QWidget()
        plot_widget.setLayout(plot_layout)

        plot_scroll_area = QScrollArea()
        plot_scroll_area.setWidget(plot_widget)
        plot_scroll_area.setWidgetResizable(True)

        main_layout.addWidget(plot_scroll_area)
        self.setLayout(main_layout)
        
        self.setWindowTitle('DAQmx Analog IO Controller')
        self.showMaximized()
        # Add reset buttons for analog output graphs
        for i, ao_channel in enumerate(self.ao_channels):
            ao_reset_button = QPushButton("Reset")
            ao_reset_button.clicked.connect(partial(self.reset_ao_graph, ao_channel))
            control_layout.addWidget(ao_reset_button, i, 11)  # Change the column to 7

        # Add reset buttons for analog input graphs
        for i, ai_channel in enumerate(self.ai_channels):
            ai_reset_button = QPushButton("Reset")
            ai_reset_button.clicked.connect(partial(self.reset_ai_graph, ai_channel))
            control_layout.addWidget(ai_reset_button, i + len(self.ao_channels), 11)
        
        # Add read analog output button and digit display for each channel
        self.ao_read_labels = []
        self.ao_read_labels = []
        for i, ao_channel in enumerate(self.ao_channels):
            ao_read_label = QLabel("0.0")
            self.ao_read_labels.append(ao_read_label)
            control_layout.addWidget(ao_read_label, i, 7)  # Change the column to 7
            
    def save_ai_data(self, channel):
        index = self.ai_channels.index(channel)
        if len(self.ai_timestamps[index]) == 0 or len(self.ai_values[index]) == 0:
            QMessageBox.warning(self, "Warning", "No data available to save for this channel.")
            return

        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getSaveFileName(self, "Save AI Data", "", "Text Files (*.txt)")
        if file_path:
            try:
                with open(file_path, 'w') as f:
                    f.write("Timestamp (s),Voltage (V)\n")
                    for t, v in zip(self.ai_timestamps[index], self.ai_values[index]):
                        f.write(f"{t:.6f},{v:.6f}\n")
                QMessageBox.information(self, "Success", f"AI data for {channel} saved successfully.")
            except Exception as e:
                QMessageBox.critical(self, "Error", f"Failed to save AI data: {str(e)}")   
                  
    def set_ao_value(self, channel, textbox):
        try:
            value = float(textbox.text())
            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.write(value)
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error setting AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def sweep_ao_voltage(self, channel, start_voltage, end_voltage, voltage_step, iterations):
        try:
            start = float(start_voltage.text())
            end = float(end_voltage.text())
            step = float(voltage_step.text())
            num_iterations = int(iterations.text())

            if start >= end:
                raise ValueError("Start voltage must be less than end voltage.")
            if step <= 0:
                raise ValueError("Voltage step must be greater than zero.")
            if num_iterations <= 0:
                raise ValueError("Number of iterations must be greater than zero.")

            voltages = np.arange(start, end + step, step)
            waveform = np.tile(voltages, num_iterations)

            with nidaqmx.Task() as task:
                task.ao_channels.add_ao_voltage_chan(channel)
                task.timing.cfg_samp_clk_timing(rate=1000, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                task.write(waveform, auto_start=True)

                index = self.ao_channels.index(channel)
                timestamps = np.linspace(0, len(waveform) / 1000, len(waveform))
                for i in range(len(waveform)):
                    self.ao_values[index].append(waveform[i])
                    self.ao_timestamps[index].append(timestamps[i])
                    self.ao_plots[channel].clear()
                    self.ao_plots[channel].plot(self.ao_timestamps[index], self.ao_values[index])
                    QApplication.processEvents()  # Process events to keep the UI responsive

                task.wait_until_done()
                task.stop()

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error sweeping AO voltage: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

    def set_ao_settings(self, channel):
        try:
            index = self.ao_channels.index(channel)
            period = float(self.ao_period_textboxes[index].text())
            iterations = int(self.ao_iterations_textboxes[index].text())

            if period <= 0:
                raise ValueError("Period must be greater than zero.")
            if iterations <= 0:
                raise ValueError("Iterations must be greater than zero.")

            if self.uploaded_data[channel] is None:
                raise ValueError("No data uploaded for this channel.")

            values = self.uploaded_data[channel]
            samples = len(values)
            t = np.linspace(0, period * iterations, samples * iterations)
            preview_waveform = np.tile(values, iterations)

            # Update the preview plot
            self.ao_preview_plots[channel].clear()
            self.ao_preview_plots[channel].plot(t, preview_waveform)
            self.ao_preview_plots[channel].setLabel('bottom', 'Time', units='s')

            QMessageBox.information(self, "Settings Applied", f"Settings applied for {channel}. Check the preview graph.")

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Error setting AO values: {str(e)}")
            
    def upload_ao_values(self, channel):
        try:
            file_dialog = QFileDialog()
            file_path, _ = file_dialog.getOpenFileName(self, "Open Text File", "", "Text Files (*.txt)")
            if file_path:
                worker = Worker(self._load_file, file_path, channel)
                worker.signals.result.connect(self.update_preview)
                worker.signals.error.connect(self.handle_error)
                self.threadpool.start(worker)
        except Exception as e:
            self.handle_error(("Error", e, ""))

    def _load_file(self, file_path, channel):
        with open(file_path, "r") as file:
            values = [float(line.strip()) for line in file.readlines()]
        return channel, values

    def update_preview(self, result):
        channel, values = result
        self.uploaded_data[channel] = values
        index = self.ao_channels.index(channel)
        
        # Update preview plot
        self.ao_preview_plots[channel].clear()
        self.ao_preview_plots[channel].plot(range(len(values)), values)
        
        # Enable the "Run" button
        self.run_buttons[index].setEnabled(True)
    
    def closeEvent(self, event):
        # Ensure all tasks are closed and resources are released
        for task in nidaqmx.system.System().tasks:
            try:
                task.close()
            except:
                pass
        event.accept()

    def run_ao_values(self, channel):
        if self.uploaded_data[channel] is None:
            QMessageBox.warning(self, "Warning", "No data uploaded for this channel.")
            return
        
        index = self.ao_channels.index(channel)
        values = self.uploaded_data[channel]
        period = float(self.ao_period_textboxes[index].text())
        iterations = int(self.ao_iterations_textboxes[index].text())

        sample_rate = len(values) / period
        samples_per_channel = len(values) * iterations

        waveform = np.tile(values, iterations)

        try:
            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO task
                ao_task.ao_channels.add_ao_voltage_chan(channel)
                ao_task.timing.cfg_samp_clk_timing(rate=sample_rate,
                                                sample_mode=AcquisitionType.FINITE,
                                                samps_per_chan=samples_per_channel)

                # Configure AI task
                ai_channel = f"Dev1/ai{channel[-1]}"
                terminal_config = getattr(TerminalConfiguration, self.ai_terminal_configs[index].currentText())
                min_val = float(self.ai_min_voltages[index].text())
                max_val = float(self.ai_max_voltages[index].text())
                
                ai_task.ai_channels.add_ai_voltage_chan(ai_channel,
                                                        terminal_config=terminal_config,
                                                        min_val=min_val,
                                                        max_val=max_val)
                ai_task.timing.cfg_samp_clk_timing(rate=sample_rate,
                                                source="ao/SampleClock",
                                                sample_mode=AcquisitionType.FINITE,
                                                samps_per_chan=samples_per_channel,
                                                active_edge=Edge.RISING)

                # Prepare the writer and reader
                ao_writer = AnalogMultiChannelWriter(ao_task.out_stream)
                ai_reader = AnalogMultiChannelReader(ai_task.in_stream)

                # Write data to AO buffer
                ao_writer.write_many_sample(waveform.reshape(1, -1))

                # Prepare buffer for AI data
                ai_data = np.zeros((1, samples_per_channel))

                # Start tasks
                ai_task.start()
                start_time = time.perf_counter()
                ao_task.start()

                # Read data
                ai_reader.read_many_sample(ai_data, number_of_samples_per_channel=samples_per_channel,
                                        timeout=period * iterations + 5.0)

                end_time = time.perf_counter()

                # Clip AI data to specified range
                ai_data = np.clip(ai_data, min_val, max_val)

            # Generate timestamps
            timestamps = np.linspace(0, period * iterations, samples_per_channel)

            # Update plots
            self.update_plot(channel, timestamps, waveform, timestamps, ai_data[0])

            # Calculate and display timing information
            expected_duration = period * iterations
            actual_duration = end_time - start_time
            timing_error = abs(actual_duration - expected_duration)

            info_message = (f"Waveform output and input completed for channel {channel}\n"
                            f"Expected duration: {expected_duration:.6f}s\n"
                            f"Actual duration: {actual_duration:.6f}s\n"
                            f"Timing error: {timing_error:.6f}s")
            
            QMessageBox.information(self, "Success", info_message)

        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error during AO/AI operation: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")
        
    def update_plots(self, channel, ao_timestamps, ao_values, ai_timestamps, ai_values):
        try:
            # Update AO plot
            self.ao_plots[channel].clear()
            self.ao_plots[channel].plot(ao_timestamps, ao_values)
            
            # Update AI plot
            ai_channel = f"Dev1/ai{self.ao_channels.index(channel)}"
            self.ai_plots[ai_channel].clear()
            self.ai_plots[ai_channel].plot(ai_timestamps, ai_values)
            
            QApplication.processEvents()
        except Exception as e:
            print(f"Error updating plots: {str(e)}")
    
    def _output_and_read_values(self, channel, waveform, timestamps, total_duration):
        try:
            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO task
                ao_task.ao_channels.add_ao_voltage_chan(channel)
                ao_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                
                # Configure AI task
                ai_channel = f"Dev1/ai{self.ao_channels.index(channel)}"
                ai_task.ai_channels.add_ai_voltage_chan(ai_channel)
                ai_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
                
                # Set up synchronization
                ai_task.triggers.start_trigger.cfg_dig_edge_start_trig(f"/{channel}/StartTrigger")
                
                # Write AO data
                ao_writer = AnalogMultiChannelWriter(ao_task.out_stream)
                ao_writer.write_many_sample(np.array([waveform]))
                
                # Prepare AI reader
                ai_reader = AnalogMultiChannelReader(ai_task.in_stream)
                ai_data = np.zeros((1, len(waveform)))
                
                # Start tasks
                ai_task.start()
                ao_task.start()
                
                # Read and update plots
                for i in range(0, len(waveform), 1000):
                    if ao_task.is_task_done():
                        break
                    progress = int((i / len(waveform)) * 100)
                    self.signals.progress.emit(progress)
                    
                    ai_reader.read_many_sample(ai_data[:, i:i+1000], number_of_samples_per_channel=min(1000, len(waveform)-i))
                    self.signals.result.emit((timestamps[:i+1000], waveform[:i+1000], ai_data[0, :i+1000]))
                    time.sleep(0.1)  # Adjust this value to control update frequency
                
                ao_task.wait_until_done(timeout=total_duration + 5.0)
                ai_task.wait_until_done(timeout=total_duration + 5.0)
        except Exception as e:
            self.signals.error.emit(str(e))

    def update_ao_plot(self, channel, timestamps, values):
        self.ao_plots[channel].clear()
        self.ao_plots[channel].plot(timestamps * 1000, values)  # Convert timestamps to milliseconds for display
        self.ao_plots[channel].setLabel('bottom', 'Time', units='ms')
        self.ao_plots[channel].setXRange(0, timestamps[-1] * 1000)  # Ensure full range is visible
        QApplication.processEvents()
        
    def handle_thread_error(self, error_message):
        QMessageBox.critical(self, "Error", f"An error occurred: {error_message}")

    def output_finished(self, channel):
        QMessageBox.information(self, "Success", f"Waveform output completed for channel {channel}")
        
    def _output_ao_values(self, channel):
        index = self.ao_channels.index(channel)
        values = self.uploaded_data[channel]
        period = float(self.ao_period_textboxes[index].text())
        iterations = int(self.ao_iterations_textboxes[index].text())

        waveform = np.tile(values, iterations)
        total_duration = period * iterations * len(values)

        with nidaqmx.Task() as task:
            task.ao_channels.add_ao_voltage_chan(channel)
            task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))
            task.write(waveform, auto_start=True)
            task.wait_until_done(timeout=total_duration + 5.0)

    def output_finished(self, channel):
        logging.debug(f"Output finished for channel {channel}")
        QMessageBox.information(self, "Success", f"Output completed for channel {channel}")

    def handle_error(self, error_info):
        QMessageBox.critical(self, "Error", str(error_info[1]))
                
    def output_ao_values(self, channel):
        try:
            index = self.ao_channels.index(channel)
            file_path = self.ao_file_paths[index]
            period = float(self.ao_period_textboxes[index].text())
            iterations = int(self.ao_iterations_textboxes[index].text())

            if file_path is None:
                raise ValueError("No file uploaded for this channel.")

            with open(file_path, "r") as file:
                values = [float(line.strip()) for line in file.readlines()]

            # Prepare the waveform data
            waveform = np.tile(values, iterations)
            total_duration = period * iterations * len(values)

            self.progress_bar.setVisible(True)
            self.progress_bar.setValue(0)

            # Create tasks for both output and input
            with nidaqmx.Task() as ao_task, nidaqmx.Task() as ai_task:
                # Configure AO task
                ao_task.ao_channels.add_ao_voltage_chan(channel)
                ao_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))

                # Configure AI task
                ai_channel = f"Dev1/ai{index}"
                ai_task.ai_channels.add_ai_voltage_chan(
                    ai_channel,
                    terminal_config=getattr(TerminalConfiguration, self.ai_terminal_configs[index].currentText()),
                    min_val=float(self.ai_min_voltages[index].text()),
                    max_val=float(self.ai_max_voltages[index].text())
                )
                ai_task.timing.cfg_samp_clk_timing(rate=len(waveform)/total_duration, sample_mode=AcquisitionType.FINITE, samps_per_chan=len(waveform))

                data_thread = DataProcessingThread(channel, waveform, total_duration, ai_task, ao_task)
                data_thread.update_progress.connect(self.progress_bar.setValue)
                data_thread.update_plot.connect(self.update_plot)
                data_thread.finished.connect(lambda: self.progress_bar.setVisible(False))
                data_thread.start()

                # Wait for the thread to finish
                data_thread.wait()

        except ValueError as e:
            QMessageBox.critical(self, "Error", str(e))
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error outputting AO values: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")
        finally:
            self.progress_bar.setVisible(False)
    
    def handle_thread_error(self, error_message):
        QMessageBox.critical
    
    def cleanup_tasks(self, ao_task, ai_task):
        try:
            ao_task.close()
            ai_task.close()
        except Exception as e:
            print(f"Error during task cleanup: {str(e)}")
        finally:
            self.progress_bar.setVisible(False)
            
    def update_progress(self, value):
        self.progress_bar.setValue(value)

    def read_ai_value(self, channel, label, index):
        try:
            task = nidaqmx.Task()
            
            terminal_config = getattr(TerminalConfiguration, self.ai_terminal_configs[index].currentText())
            min_val = float(self.ai_min_voltages[index].text())
            max_val = float(self.ai_max_voltages[index].text())
            sample_rate = float(self.ai_sampling_rates[index].text())

            task.ai_channels.add_ai_voltage_chan(
                channel,
                terminal_config=terminal_config,
                min_val=min_val,
                max_val=max_val
            )

            # Set up timing for finite acquisition
            samples_to_read = 1000  # Number of samples to read each time
            task.timing.cfg_samp_clk_timing(
                rate=sample_rate,
                sample_mode=AcquisitionType.FINITE,
                samps_per_chan=samples_to_read
            )

            def update_ai_plot():
                try:
                    # Read data
                    data = task.read(number_of_samples_per_channel=samples_to_read)
                    data = np.clip(data, min_val, max_val)
                    
                    label.setText(f"{data[-1]:.2f}")  # Display the most recent value

                    # Generate timestamps
                    duration = samples_to_read / sample_rate
                    timestamps = np.linspace(0, duration, samples_to_read)

                    # Update plot
                    self.update_ai_plot(channel, timestamps, data)

                    # Restart the task for the next acquisition
                    task.stop()
                    task.start()

                except Exception as e:
                    print(f"Error updating AI plot: {str(e)}")

            timer = pg.QtCore.QTimer()
            timer.timeout.connect(update_ai_plot)
            update_interval = max(100, int(1000 * samples_to_read / sample_rate))  # Update at most 10 times per second
            timer.start(update_interval)

        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error reading AI value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")
    
    def update_plot(self, channel, ao_timestamps, ao_values, ai_timestamps, ai_values):
        index = self.ao_channels.index(channel)
        
        # Update AO plot
        self.ao_values[index] = ao_values
        self.ao_timestamps[index] = ao_timestamps
        self.ao_plots[channel].clear()
        self.ao_plots[channel].plot(ao_timestamps * 1000, ao_values)  # Convert to milliseconds
        self.ao_plots[channel].setLabel('bottom', 'Time', units='ms')
        self.ao_plots[channel].setXRange(0, ao_timestamps[-1] * 1000)
        
        # Update AI plot
        ai_channel = f"Dev1/ai{index}"
        self.ai_values[index] = ai_values
        self.ai_timestamps[index] = ai_timestamps
        self.ai_plots[ai_channel].clear()
        self.ai_plots[ai_channel].plot(ai_timestamps * 1000, ai_values)  # Convert to milliseconds
        self.ai_plots[ai_channel].setLabel('bottom', 'Time', units='ms')
        self.ai_plots[ai_channel].setXRange(0, ai_timestamps[-1] * 1000)
        
        QApplication.processEvents()

                            
    def reset_ao_graph(self, channel):
        index = self.ao_channels.index(channel)
        self.ao_values[index] = np.array([])
        self.ao_timestamps[index] = np.array([])
        self.ao_plots[channel].clear()

    def reset_ai_graph(self, channel):
        index = self.ai_channels.index(channel)
        self.ai_values[index] = np.array([])
        self.ai_timestamps[index] = np.array([])
        self.ai_plots[channel].clear()
    
    def read_ao_value(self, channel, index):
        try:
            with nidaqmx.Task() as task:
                task.ai_channels.add_ai_voltage_chan(channel)  # Change to ai_channels
                value = task.read()
                self.ao_read_labels[index].setText(f"{value[0]:.2f}")
        except nidaqmx.errors.DaqError as e:
            QMessageBox.critical(self, "DAQ Error", f"Error reading AO value: {str(e)}")
        except Exception as e:
            QMessageBox.critical(self, "Error", f"Unexpected error: {str(e)}")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = DAQmxController()
    sys.exit(app.exec_())

In [ ]:
# After sweep combine file
# Combine 2 text file
import numpy as np

def combine_channel_files(file_paths, output_file, num_channels):
    # Read data from each file
    channel_data = []
    for file_path in file_paths:
        with open(file_path, 'r') as f:
            # Skip header if present
            first_line = f.readline().strip()
            if not is_float(first_line.split(',')[0]):  # Check if the first item is not a float
                print(f"Skipping header in {file_path}: {first_line}")
            else:
                f.seek(0)  # If no header, go back to the start of the file
            
            # Read data, handling both comma-separated and space/tab-separated formats
            data = []
            for line in f:
                try:
                    # Try comma-separated first
                    values = [float(val.strip()) for val in line.split(',') if val.strip()]
                    if len(values) == 0:
                        continue
                    if len(values) > 1:
                        data.append(values[-1])  # Take the last value if multiple columns
                    else:
                        data.append(values[0])
                except ValueError:
                    # If comma-separated fails, try space/tab-separated
                    try:
                        values = [float(val.strip()) for val in line.split() if val.strip()]
                        if len(values) > 0:
                            data.append(values[-1])  # Take the last value if multiple columns
                    except ValueError:
                        print(f"Skipping invalid line in {file_path}: {line.strip()}")
            
            channel_data.append(np.array(data))
    
    # Ensure all channels have the same number of samples
    min_samples = min(len(data) for data in channel_data)
    channel_data = [data[:min_samples] for data in channel_data]
    
    # Pad with zeros if we have fewer channels than specified
    while len(channel_data) < num_channels:
        channel_data.append(np.zeros(min_samples))
    
    # Combine data into a 2D array
    combined_data = np.column_stack(channel_data[:num_channels])
    
    # Save combined data to a new text file
    with open(output_file, 'w') as f:
        f.write("Channel 1\tChannel 2\n")  # Write header
        for row in combined_data:
            f.write('\t'.join(f'{val:.6f}' for val in row) + '\n')
    
    print(f"Combined data saved to {output_file}")
    print(f"Shape of combined data: {combined_data.shape}")

def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# Usage
file_paths = [
    'voltage_sweep.txt',
    '1x10pwr2.txt'
]
output_file = 'Voltage sweep_voltage receive.txt'
NUM_CHANNELS = 2  # Changed to 2 to include both input files

combine_channel_files(file_paths, output_file, NUM_CHANNELS)